In [138]:
# Import libraries - REQUIRES pip version 9.0.3
import pandas
import os
from os.path import join
import sys
import scipy.stats
import numpy
import math
import pickle
import copy
import time
import warnings
import gc

#Simplify reading/writing files
cwd=os.path.realpath(os.path.join(os.path.dirname(os.getcwd()),"..",".."))

# Using Cobrapy 0.13.0
import cobra
import cobra.test
import cobra.flux_analysis.gapfilling
from cobra import Reaction, Metabolite
from cobra.io import write_sbml_model
from cobra.flux_analysis import  flux_variability_analysis
from cobra.flux_analysis.reaction import assess_component
from cobra.manipulation.delete import *
from cobra.flux_analysis.parsimonious import add_pfba
from cobra.medium import find_boundary_types
from cobra.util import solver as sutil



In [139]:
# Quicker way to read in models
import pickle
def read_model(fileName, obj='none'):
    
    fileType = fileName.split('.')[-1]
    
    if fileType == 'sbml' or fileType == 'xml':
        model = cobra.io.read_sbml_model(fileName)
    elif fileType == 'json':
        model = cobra.io.load_json_model(fileName)
    elif fileType == 'yaml':
        model = cobra.io.load_yaml_model(fileName)
    elif fileType == 'mat':
        model = cobra.io.load_matlab_model(fileName)
    elif fileType == 'pkl':
        model = pickle.load(open(fileName, 'rb'))
    else:
        raise TypeError('Unrecognized file extension')
    
    if obj != 'none': model.objective = obj
    for rxn in model.boundary: rxn.bounds = (-1000., 1000.)
        
    return model

In [140]:
#Set medias
rpmi_req=set(['EX_apoACP_c_', 'EX_trdrd_c_','EX_cobalt2_e_','EX_cu2_e_','EX_fe3_e_','EX_mn2_e_','EX_mobd_e_','EX_zn2_e_','EX_ca2_e_','EX_no3_e_','EX_h2o_e_','EX_k_e_','EX_cl_e_','EX_mg2_e_','EX_so4_e_','EX_na1_e_','EX_pi_e_','EX_h_e_','EX_arg_L_e_','EX_asn_L_e_','EX_asp_L_e_','EX_cys_L_e_','EX_glu_L_e_','EX_gly_e_','EX_his_L_e_','EX_ile_L_e_','EX_leu_L_e_','EX_lys_L_e_','EX_met_L_e_','EX_phe_L_e_','EX_pro_L_e_','EX_ser_L_e_','EX_thr_L_e_','EX_trp_L_e_','EX_tyr_L_e_','EX_val_L_e_','EX_pnto_R_e_','EX_chol_e_','EX_inost_e_','EX_glc_D_e_','EX_gthrd_e_','EX_co2_e_'])
mdm_req=set(['EX_apoACP_c_', 'EX_trdrd_c_','EX_co2_e_', 'EX_cobalt2_e_', 'EX_cu2_e_' , 'EX_h_e_', 'EX_h2o_e_','EX_mn2_e_','EX_mobd_e_','EX_tungs_e_', 'EX_zn2_e_','EX_na1_e_','EX_cl_e_','EX_k_e_','EX_so4_e_','EX_nh4_e_','EX_mg2_e_','EX_pi_e_','EX_ca2_e_','EX_fe3_e_','EX_no3_e_','EX_asp_L_e_','EX_glu_L_e_','EX_arg_L_e_','EX_gly_e_','EX_ser_L_e_','EX_leu_L_e_','EX_ile_L_e_','EX_val_L_e_','EX_tyr_L_e_','EX_cys_L_e_','EX_pro_L_e_','EX_trp_L_e_','EX_thr_L_e_','EX_phe_L_e_','EX_asn_L_e_','EX_gln_L_e_','EX_his_L_e_','EX_met_L_e_','EX_ala_L_e_','EX_lys_L_e_','EX_gthrd_e_','EX_thm_e_','EX_pnto_R_e_','EX_glc_D_e_','EX_hxan_e_','EX_ura_e_'])

def rpmi(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in rpmi_req:
            reaction.lower_bound = -20.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -20.

def complete(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=-20.

def mdm(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in mdm_req:
            reaction.lower_bound = -20.
        elif reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.
        elif reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -20.

def rpmi_molarity(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in rpmi_req:
            reaction.lower_bound = -20. 
        if 'EX_glc_D_e_' in reaction.id:
            reaction.lower_bound = -11.1012
        if 'EX_ser_L_e_' in reaction.id:
            reaction.lower_bound = -0.2855
        if 'EX_asp_L_e_' in reaction.id:
            reaction.lower_bound = -0.1503
        if 'EX_asn_L_e_' in reaction.id:
            reaction.lower_bound = -0.3784
        if 'EX_glu_L_e_' in reaction.id:
            reaction.lower_bound = -0.1359
        if 'EX_gly_e_' in reaction.id:
            reaction.lower_bound = -0.1332
        if 'EX_val_L_e_' in reaction.id:
            reaction.lower_bound = -0.1707
        if 'EX_leu_L_e_' in reaction.id:
            reaction.lower_bound = -0.3812
        if 'EX_pro_L_e_' in reaction.id:
            reaction.lower_bound = -0.1737
        if 'EX_phe_L_e_' in reaction.id:
            reaction.lower_bound = -0.0908
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.219
        if 'EX_ca2_e_' in reaction.id:
            reaction.lower_bound = -0.424
        if 'EX_no3_e_' in reaction.id:
            reaction.lower_bound = -0.847
        if 'EX_k_e_' in reaction.id:
            reaction.lower_bound = -5.3655
        if 'EX_cl_e_' in reaction.id:
            reaction.lower_bound = -108.6993
        if 'EX_mg2_e_' in reaction.id:
            reaction.lower_bound = -0.4056
        if 'EX_so4_e_' in reaction.id:
            reaction.lower_bound = -0.4056
        if 'EX_na1_e_' in reaction.id:
            reaction.lower_bound = -137.9818
        if 'EX_pi_e_' in reaction.id:
            reaction.lower_bound = -6.354
        if 'EX_arg_L_e_' in reaction.id:
            reaction.lower_bound = -1.1481
        if 'EX_cys_L_e_' in reaction.id:
            reaction.lower_bound = -10.416
        if 'EX_his_L_e_' in reaction.id:
            reaction.lower_bound = -0.0967
        if 'EX_ile_L_e_' in reaction.id:
            reaction.lower_bound = -0.3812
        if 'EX_met_L_e_' in reaction.id:
            reaction.lower_bound = -0.1005
        if 'EX_thr_L_e_' in reaction.id:
            reaction.lower_bound = -0.1679
        if 'EX_trp_L_e_' in reaction.id:
            reaction.lower_bound = -0.0245
        if 'EX_lys_L_e_' in reaction.id:
            reaction.lower_bound = -0.219
        if 'EX_tyr_L_e_' in reaction.id:
            reaction.lower_bound = -0.1104
        if 'EX_pnto_R_e_' in reaction.id:
            reaction.lower_bound = -0.001
        if 'EX_chol_e_' in reaction.id:
            reaction.lower_bound = -0.0215
        if 'EX_inost_e_' in reaction.id:
            reaction.lower_bound = -0.1943
        if 'EX_gthrd_e_' in reaction.id:
            reaction.lower_bound = -0.0033
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -20.

In [141]:
# Read in  model
import cobra

model=read_model(cwd+'/Gc_GENRE_2022/Curate_Gc_Model/In_progress_curation/NGO_557.sbml')
model



Name,iNGO_557
Memory address,0x01487834f400
Number of metabolites,1341
Number of reactions,1433
Number of groups,0
Objective expression,1.0*Nm_Ess_biomass - 1.0*Nm_Ess_biomass_reverse_46af5
Compartments,"periplasm, cytosol, extracellular"


In [142]:
#Count number of imbalances
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv(cwd+ '/Gc_GENRE_2022/Curate_Gc_Model/totalimbalances.csv')
mass_charge

291 291 291


,Reactions,Charge,Masses
0,3OAS161a,,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
1,ASNSYN,,"{'H': 2.0, 'O': 2.0}"
2,BTS4,,{'S': 1.0}
3,CYTCS,,"{'C': 8.0, 'H': 14.0, 'N': 4.0, 'O': 4.0, 'S':..."
4,DASYN161a,,{'H': -1.0}
...,...,...,...
286,CPH4S,,"{'C': -7.0, 'H': -8.0, 'N': -5.0, 'O': -3.0}"
287,CDGS,,"{'H': 3.0, 'N': 1.0}"
288,CDGR,,{'H': -5.0}
289,CCGS,,"{'N': -1.0, 'O': 2.0}"


In [143]:
#Several imbalances are due to missing formulas. 
#Find missing formulas
no_formula = ("")
for metabolites in model.metabolites:
        if metabolites.formula == no_formula:
            print (metabolites.id)

essbiomass
protein
pepglycan_NM_p
capminusbiomass
lipid_NM_p
esslipid_NM_p
rna_c
biomass
dna_c
6pthp_c
cph4_c
cdg_c
preq0_c
preq1_c


In [144]:
#Many Imbalances are due to lack of charges. Adding charges from the Bigg Ecoli model (what the NM model was based on)
ecoli = read_model(cwd+ '/Gc_GENRE_2022/Models/ecoli.json')

In [145]:
#for x in model.metabolites:
#    if x in ecoli.metabolites:
#        x.charge=ecoli.metabolites.get_by_id(str(x)).charge
#    else:
#        pass

In [146]:
model.metabolites.get_by_id('2agpg161_c').charge=-1
model.metabolites.get_by_id('2agpg161_p').charge=-1
model.metabolites.get_by_id('2agpg180_c').charge=-1
model.metabolites.get_by_id('2agpg180_p').charge=-1
model.metabolites.get_by_id('2agpg181_c').charge=-1
model.metabolites.get_by_id('2agpg181_p').charge=-1
model.metabolites.get_by_id('2ahbut_c').charge=-1
model.metabolites.get_by_id('2amsa_c').charge=0
model.metabolites.get_by_id('2aobut_c').charge=0
model.metabolites.get_by_id('2cpr5p_c').charge=-3
model.metabolites.get_by_id('2dda7p_c').charge=-3
model.metabolites.get_by_id('2ddecg3p_c').charge=-1
model.metabolites.get_by_id('2ddecg3p_p').charge=-1
model.metabolites.get_by_id('2ddg6p_c').charge=-3
model.metabolites.get_by_id('2ddglcn_e').charge=-1
model.metabolites.get_by_id('2ddglcn_p').charge=-1
model.metabolites.get_by_id('2dh3dgal6p_c').charge=-3
model.metabolites.get_by_id('12dgr181_c').charge=0
model.metabolites.get_by_id('10fthf_c').charge=-2
model.metabolites.get_by_id('12dgr120_c').charge=0
model.metabolites.get_by_id('2dhp_c').charge=-1
model.metabolites.get_by_id('12dgr181_p').charge=0
model.metabolites.get_by_id('12dgr120_p').charge=0
model.metabolites.get_by_id('12dgr140_c').charge=0
model.metabolites.get_by_id('12dgr140_p').charge=0
model.metabolites.get_by_id('12dgr141_c').charge=0
model.metabolites.get_by_id('12dgr141_p').charge=0
model.metabolites.get_by_id('12dgr160_c').charge=0
model.metabolites.get_by_id('12dgr160_p').charge=0
model.metabolites.get_by_id('2hdecg3p_c').charge=-1
model.metabolites.get_by_id('13dpg_c').charge=-4
model.metabolites.get_by_id('12dgr161_c').charge=0
model.metabolites.get_by_id('2hdecg3p_p').charge=-1
model.metabolites.get_by_id('12dgr161_p').charge=0
model.metabolites.get_by_id('2ippm_c').charge=-2
model.metabolites.get_by_id('12dgr180_c').charge=0
model.metabolites.get_by_id('12dgr180_p').charge=0
model.metabolites.get_by_id('2mahmp_c').charge=-3
model.metabolites.get_by_id('15dap_e').charge=2
model.metabolites.get_by_id('15dap_p').charge=2
model.metabolites.get_by_id('2mcit_c').charge=-3
model.metabolites.get_by_id('2me4p_c').charge=-2
model.metabolites.get_by_id('2mecdp_c').charge=-2
model.metabolites.get_by_id('2obut_c').charge=-1
model.metabolites.get_by_id('3hoctaACP_c').charge=-1
model.metabolites.get_by_id('2odecg3p_c').charge=-1
model.metabolites.get_by_id('2odecg3p_p').charge=-1
model.metabolites.get_by_id('3hpalmACP_c').charge=-1
model.metabolites.get_by_id('2ohph_c').charge=0
model.metabolites.get_by_id('3hpppn_e').charge=-1
model.metabolites.get_by_id('3hpppn_p').charge=-1
model.metabolites.get_by_id('2ombzl_c').charge=0
model.metabolites.get_by_id('2omhmbl_c').charge=0
model.metabolites.get_by_id('1ddecg3p_c').charge=-2
model.metabolites.get_by_id('3ig3p_c').charge=-2
model.metabolites.get_by_id('3mob_c').charge=-1
model.metabolites.get_by_id('2ommbl_c').charge=0
model.metabolites.get_by_id('3mop_c').charge=-1
model.metabolites.get_by_id('2omph_c').charge=0
model.metabolites.get_by_id('1hdec9eg3p_c').charge=-2
model.metabolites.get_by_id('3ocddec5eACP_c').charge=-1
model.metabolites.get_by_id('2oph_c').charge=0
model.metabolites.get_by_id('1hdecg3p_c').charge=-2
model.metabolites.get_by_id('2p4c2me_c').charge=-4
model.metabolites.get_by_id('1odec11eg3p_c').charge=-2
model.metabolites.get_by_id('2pg_c').charge=-3
model.metabolites.get_by_id('1odecg3p_c').charge=-2
model.metabolites.get_by_id('3ocmrs7eACP_c').charge=-1
model.metabolites.get_by_id('3ocpalm9eACP_c').charge=-1
model.metabolites.get_by_id('1pyr5c_c').charge=-1
model.metabolites.get_by_id('2pglyc_c').charge=-3
model.metabolites.get_by_id('1tdec7eg3p_c').charge=-2
model.metabolites.get_by_id('2tdec7eg3p_c').charge=-1
model.metabolites.get_by_id('1tdecg3p_c').charge=-2
model.metabolites.get_by_id('3oddecACP_c').charge=-1
model.metabolites.get_by_id('2tdec7eg3p_p').charge=-1
model.metabolites.get_by_id('3odecACP_c').charge=-1
model.metabolites.get_by_id('2tdecg3p_c').charge=-1
model.metabolites.get_by_id('2tdecg3p_p').charge=-1
model.metabolites.get_by_id('3ohexACP_c').charge=-1
model.metabolites.get_by_id('3omrsACP_c').charge=-1
model.metabolites.get_by_id('34dhpac_e').charge=0
model.metabolites.get_by_id('3ooctACP_c').charge=-1
model.metabolites.get_by_id('34dhpac_p').charge=0
model.metabolites.get_by_id('3ooctdACP_c').charge=-1
model.metabolites.get_by_id('34hpp_c').charge=-1
model.metabolites.get_by_id('3amp_e').charge=-2
model.metabolites.get_by_id('3opalmACP_c').charge=-1
model.metabolites.get_by_id('3ophb_c').charge=-1
model.metabolites.get_by_id('3amp_p').charge=-2
model.metabolites.get_by_id('3c2hmp_c').charge=-2
model.metabolites.get_by_id('3pg_c').charge=-3
model.metabolites.get_by_id('3c3hmp_c').charge=-2
model.metabolites.get_by_id('3php_c').charge=-3
model.metabolites.get_by_id('3psme_c').charge=-4
model.metabolites.get_by_id('3c4mop_c').charge=-2
model.metabolites.get_by_id('3cmp_e').charge=-2
model.metabolites.get_by_id('3cmp_p').charge=-2
model.metabolites.get_by_id('3ump_e').charge=-2
model.metabolites.get_by_id('3ump_p').charge=-2
model.metabolites.get_by_id('4abut_e').charge=0
model.metabolites.get_by_id('23dhdp_c').charge=-2
model.metabolites.get_by_id('23dhmb_c').charge=-1
model.metabolites.get_by_id('23dhmp_c').charge=-1
model.metabolites.get_by_id('4abut_p').charge=0
model.metabolites.get_by_id('4abz_c').charge=-1
model.metabolites.get_by_id('3dhq_c').charge=-1
model.metabolites.get_by_id('4adcho_c').charge=-1
model.metabolites.get_by_id('3dhsk_c').charge=-1
model.metabolites.get_by_id('25aics_c').charge=-4
model.metabolites.get_by_id('4ahmmp_c').charge=0
model.metabolites.get_by_id('4ampm_c').charge=-2
model.metabolites.get_by_id('4c2me_c').charge=-2
model.metabolites.get_by_id('25drapp_c').charge=-2
model.metabolites.get_by_id('3haACP_c').charge=-1
model.metabolites.get_by_id('26dap_LL_c').charge=0
model.metabolites.get_by_id('3hcddec5eACP_c').charge=-1
model.metabolites.get_by_id('4hba_c').charge=0
model.metabolites.get_by_id('4hbz_c').charge=-1
model.metabolites.get_by_id('3hcinnm_e').charge=-1
model.metabolites.get_by_id('2agpe120_c').charge=0
model.metabolites.get_by_id('4hoxpacd_e').charge=0
model.metabolites.get_by_id('3hcinnm_p').charge=-1
model.metabolites.get_by_id('4hoxpacd_p').charge=0
model.metabolites.get_by_id('2agpe120_p').charge=0
model.metabolites.get_by_id('4hthr_c').charge=0
model.metabolites.get_by_id('3hcmrs7eACP_c').charge=-1
model.metabolites.get_by_id('2agpe140_c').charge=0
model.metabolites.get_by_id('3hcpalm9eACP_c').charge=-1
model.metabolites.get_by_id('2agpe140_p').charge=0
model.metabolites.get_by_id('4mop_c').charge=-1
model.metabolites.get_by_id('4mpetz_c').charge=-2
model.metabolites.get_by_id('2agpe141_c').charge=0
model.metabolites.get_by_id('2agpe141_p').charge=0
model.metabolites.get_by_id('2agpe160_c').charge=0
model.metabolites.get_by_id('4pasp_c').charge=-2
model.metabolites.get_by_id('4per_c').charge=-3
model.metabolites.get_by_id('4ppan_c').charge=-3
model.metabolites.get_by_id('2agpe160_p').charge=0
model.metabolites.get_by_id('3hddecACP_c').charge=-1
model.metabolites.get_by_id('2agpe161_c').charge=0
model.metabolites.get_by_id('4ppcys_c').charge=-3
model.metabolites.get_by_id('3hdecACP_c').charge=-1
model.metabolites.get_by_id('4r5au_c').charge=0
model.metabolites.get_by_id('2agpe161_p').charge=0
model.metabolites.get_by_id('2agpe180_c').charge=0
model.metabolites.get_by_id('2agpe180_p').charge=0
model.metabolites.get_by_id('5aizc_c').charge=-3
model.metabolites.get_by_id('5aop_c').charge=0
model.metabolites.get_by_id('5aprbu_c').charge=-2
model.metabolites.get_by_id('2agpe181_c').charge=0
model.metabolites.get_by_id('3hhexACP_c').charge=-1
model.metabolites.get_by_id('2agpe181_p').charge=0
model.metabolites.get_by_id('5apru_c').charge=-2
model.metabolites.get_by_id('3hmrsACP_c').charge=-1
model.metabolites.get_by_id('2agpg120_c').charge=-1
model.metabolites.get_by_id('2agpg120_p').charge=-1
model.metabolites.get_by_id('5caiz_c').charge=-3
model.metabolites.get_by_id('3hoctACP_c').charge=-1
model.metabolites.get_by_id('2agpg140_c').charge=-1
model.metabolites.get_by_id('2agpg140_p').charge=-1
model.metabolites.get_by_id('5dglcn_e').charge=-1
model.metabolites.get_by_id('5dglcn_p').charge=-1
model.metabolites.get_by_id('5drib_c').charge=0
model.metabolites.get_by_id('2agpg141_c').charge=-1
model.metabolites.get_by_id('2agpg141_p').charge=-1
model.metabolites.get_by_id('2agpg160_c').charge=-1
model.metabolites.get_by_id('2agpg160_p').charge=-1
model.metabolites.get_by_id('acACP_c').charge=-1
model.metabolites.get_by_id('5mta_c').charge=0
model.metabolites.get_by_id('ac_c').charge=-1
model.metabolites.get_by_id('5mthf_c').charge=-1
model.metabolites.get_by_id('ac_e').charge=-1
model.metabolites.get_by_id('5mtr_c').charge=0
model.metabolites.get_by_id('ac_p').charge=-1
model.metabolites.get_by_id('6hmhpt_c').charge=0
model.metabolites.get_by_id('6hmhptpp_c').charge=-3
model.metabolites.get_by_id('acac_e').charge=-1
model.metabolites.get_by_id('6pgc_c').charge=-3
model.metabolites.get_by_id('acac_p').charge=-1
model.metabolites.get_by_id('6pgl_c').charge=-2
model.metabolites.get_by_id('acald_c').charge=0
model.metabolites.get_by_id('8aonn_c').charge=0
model.metabolites.get_by_id('acald_e').charge=0
model.metabolites.get_by_id('ACP_c').charge=-1
model.metabolites.get_by_id('acald_p').charge=0
model.metabolites.get_by_id('LalaDgluMdapDala_c').charge=-1
model.metabolites.get_by_id('LalaDgluMdapDala_e').charge=-1
model.metabolites.get_by_id('accoa_c').charge=-4
model.metabolites.get_by_id('alltn_e').charge=0
model.metabolites.get_by_id('LalaDgluMdapDala_p').charge=-1
model.metabolites.get_by_id('LalaDgluMdap_c').charge=-1
model.metabolites.get_by_id('alltn_p').charge=0
model.metabolites.get_by_id('LalaDgluMdap_e').charge=-1
model.metabolites.get_by_id('acg5p_c').charge=-3
model.metabolites.get_by_id('LalaDgluMdap_p').charge=-1
model.metabolites.get_by_id('acg5sa_c').charge=-1
model.metabolites.get_by_id('LalaDglu_c').charge=-1
model.metabolites.get_by_id('amet_c').charge=1
model.metabolites.get_by_id('acgal_e').charge=0
model.metabolites.get_by_id('Sfglutth_c').charge=-1
model.metabolites.get_by_id('ametam_c').charge=2
model.metabolites.get_by_id('acgal_p').charge=0
model.metabolites.get_by_id('amob_c').charge=0
model.metabolites.get_by_id('amp_c').charge=-2
model.metabolites.get_by_id('acgam1p_c').charge=-2
model.metabolites.get_by_id('amp_e').charge=-2
model.metabolites.get_by_id('acgam1p_e').charge=-2
model.metabolites.get_by_id('amp_p').charge=-2
model.metabolites.get_by_id('acgam1p_p').charge=-2
model.metabolites.get_by_id('anhgm3p_c').charge=-2
model.metabolites.get_by_id('ca2_c').charge=2
model.metabolites.get_by_id('ca2_e').charge=2
model.metabolites.get_by_id('anhgm3p_p').charge=-2
model.metabolites.get_by_id('acgam_c').charge=0
model.metabolites.get_by_id('anhgm4p_c').charge=-2
model.metabolites.get_by_id('ca2_p').charge=2
model.metabolites.get_by_id('acgam_e').charge=0
model.metabolites.get_by_id('anhgm4p_p').charge=-2
model.metabolites.get_by_id('acgam_p').charge=0
model.metabolites.get_by_id('anhgm_c').charge=-1
model.metabolites.get_by_id('cbasp_c').charge=-2
model.metabolites.get_by_id('anhgm_e').charge=-1
model.metabolites.get_by_id('acglu_c').charge=-2
model.metabolites.get_by_id('anhgm_p').charge=-1
model.metabolites.get_by_id('anhm3p_c').charge=-2
model.metabolites.get_by_id('anhm4p_c').charge=-2
model.metabolites.get_by_id('acmana_c').charge=0
model.metabolites.get_by_id('anhm_c').charge=-1
model.metabolites.get_by_id('cbp_c').charge=-2
model.metabolites.get_by_id('anth_c').charge=-1
model.metabolites.get_by_id('acmana_e').charge=0
model.metabolites.get_by_id('cd2_c').charge=2
model.metabolites.get_by_id('acmana_p').charge=0
model.metabolites.get_by_id('cd2_e').charge=2
model.metabolites.get_by_id('cd2_p').charge=2
model.metabolites.get_by_id('ap4a_c').charge=-4
model.metabolites.get_by_id('cddec5eACP_c').charge=-1
model.metabolites.get_by_id('acmum6p_c').charge=-3
model.metabolites.get_by_id('cdec3eACP_c').charge=-1
model.metabolites.get_by_id('acmum_e').charge=-1
model.metabolites.get_by_id('cdp_c').charge=-3
model.metabolites.get_by_id('acmum_p').charge=-1
model.metabolites.get_by_id('cdpdddecg_c').charge=-2
model.metabolites.get_by_id('acnam_c').charge=-1
model.metabolites.get_by_id('cdpdhdecg_c').charge=-2
model.metabolites.get_by_id('cdpdodecg_c').charge=-2
model.metabolites.get_by_id('cdpdtdec7eg_c').charge=-2
model.metabolites.get_by_id('cdpdtdecg_c').charge=-2
model.metabolites.get_by_id('acon_C_c').charge=-3
model.metabolites.get_by_id('acon_T_c').charge=-3
model.metabolites.get_by_id('aps_c').charge=-2
model.metabolites.get_by_id('cgly_c').charge=0
model.metabolites.get_by_id('ara5p_c').charge=-2
model.metabolites.get_by_id('acorn_c').charge=0
model.metabolites.get_by_id('cgly_e').charge=0
model.metabolites.get_by_id('acser_c').charge=0
model.metabolites.get_by_id('cgly_p').charge=0
model.metabolites.get_by_id('acser_e').charge=0
model.metabolites.get_by_id('chol_c').charge=1
model.metabolites.get_by_id('acser_p').charge=0
model.metabolites.get_by_id('chol_e').charge=1
model.metabolites.get_by_id('chol_p').charge=1
model.metabolites.get_by_id('actACP_c').charge=-1
model.metabolites.get_by_id('chor_c').charge=-2
model.metabolites.get_by_id('actp_c').charge=-2
model.metabolites.get_by_id('ade_c').charge=0
model.metabolites.get_by_id('cit_c').charge=-3
model.metabolites.get_by_id('ade_e').charge=0
model.metabolites.get_by_id('cit_e').charge=-3
model.metabolites.get_by_id('ade_p').charge=0
model.metabolites.get_by_id('cit_p').charge=-3
model.metabolites.get_by_id('adn_c').charge=0
model.metabolites.get_by_id('ckdo_c').charge=-2
model.metabolites.get_by_id('cl_c').charge=-1
model.metabolites.get_by_id('cl_e').charge=-1
model.metabolites.get_by_id('cl_p').charge=-1
model.metabolites.get_by_id('clpn120_p').charge=-2
model.metabolites.get_by_id('argsuc_c').charge=-1
model.metabolites.get_by_id('clpn140_p').charge=-2
model.metabolites.get_by_id('clpn141_p').charge=-2
model.metabolites.get_by_id('ascb6p_c').charge=-2
model.metabolites.get_by_id('clpn160_p').charge=-2
model.metabolites.get_by_id('adp_c').charge=-3
model.metabolites.get_by_id('clpn161_p').charge=-2
model.metabolites.get_by_id('clpn180_p').charge=-2
model.metabolites.get_by_id('clpn181_p').charge=-2
model.metabolites.get_by_id('cmp_c').charge=-2
model.metabolites.get_by_id('cmp_e').charge=-2
model.metabolites.get_by_id('adphep_DD_c').charge=-2
model.metabolites.get_by_id('cmp_p').charge=-2
model.metabolites.get_by_id('adphep_LD_c').charge=-2
model.metabolites.get_by_id('aso3_e').charge=-3
model.metabolites.get_by_id('co2_c').charge=0
model.metabolites.get_by_id('adprib_c').charge=-2
model.metabolites.get_by_id('co2_e').charge=0
model.metabolites.get_by_id('aso3_p').charge=-3
model.metabolites.get_by_id('ag_c').charge=1
model.metabolites.get_by_id('co2_p').charge=0
model.metabolites.get_by_id('ag_e').charge=1
model.metabolites.get_by_id('coa_c').charge=-4
model.metabolites.get_by_id('cobalt2_c').charge=2
model.metabolites.get_by_id('cobalt2_e').charge=2
model.metabolites.get_by_id('agm_c').charge=2
model.metabolites.get_by_id('cobalt2_p').charge=2
model.metabolites.get_by_id('aspsa_c').charge=0
model.metabolites.get_by_id('agm_e').charge=2
model.metabolites.get_by_id('agm_p').charge=2
model.metabolites.get_by_id('colipa_e').charge=-11
model.metabolites.get_by_id('ahcys_c').charge=0
model.metabolites.get_by_id('atp_c').charge=-4
model.metabolites.get_by_id('ahdt_c').charge=-4
model.metabolites.get_by_id('aicar_c').charge=-2
model.metabolites.get_by_id('air_c').charge=-2
model.metabolites.get_by_id('akg_c').charge=-2
model.metabolites.get_by_id('akg_e').charge=-2
model.metabolites.get_by_id('akg_p').charge=-2
model.metabolites.get_by_id('ala_B_c').charge=0
model.metabolites.get_by_id('btn_c').charge=-1
model.metabolites.get_by_id('ala_B_e').charge=0
model.metabolites.get_by_id('ala_B_p').charge=0
model.metabolites.get_by_id('but2eACP_c').charge=-1
model.metabolites.get_by_id('butACP_c').charge=-1
model.metabolites.get_by_id('cpppg3_c').charge=-4
model.metabolites.get_by_id('but_e').charge=-1
model.metabolites.get_by_id('but_p').charge=-1
model.metabolites.get_by_id('butso3_e').charge=-1
model.metabolites.get_by_id('crn_e').charge=0
model.metabolites.get_by_id('butso3_p').charge=-1
model.metabolites.get_by_id('crn_p').charge=0
model.metabolites.get_by_id('alaala_c').charge=0
model.metabolites.get_by_id('alaala_e').charge=0
model.metabolites.get_by_id('csn_c').charge=0
model.metabolites.get_by_id('alaala_p').charge=0
model.metabolites.get_by_id('csn_e').charge=0
model.metabolites.get_by_id('dopa_p').charge=1
model.metabolites.get_by_id('csn_p').charge=0
model.metabolites.get_by_id('cu_p').charge=1
model.metabolites.get_by_id('dpcoa_c').charge=-2
model.metabolites.get_by_id('cyan_e').charge=0
model.metabolites.get_by_id('ctp_c').charge=-4
model.metabolites.get_by_id('dsbcox_p').charge=0
model.metabolites.get_by_id('cyan_p').charge=0
model.metabolites.get_by_id('dsbcrd_p').charge=0
model.metabolites.get_by_id('cynt_e').charge=-1
model.metabolites.get_by_id('cu2_c').charge=2
model.metabolites.get_by_id('cu2_e').charge=2
model.metabolites.get_by_id('dsbdox_c').charge=0
model.metabolites.get_by_id('cu2_p').charge=2
model.metabolites.get_by_id('cu_c').charge=1
model.metabolites.get_by_id('cynt_p').charge=-1
model.metabolites.get_by_id('dsbdrd_c').charge=0
model.metabolites.get_by_id('cu_e').charge=1
model.metabolites.get_by_id('dsbgox_p').charge=0
model.metabolites.get_by_id('dsbgrd_p').charge=0
model.metabolites.get_by_id('dscl_c').charge=-7
model.metabolites.get_by_id('dtbt_c').charge=-1
model.metabolites.get_by_id('cytd_c').charge=0
model.metabolites.get_by_id('dtdp4d6dg_c').charge=-2
model.metabolites.get_by_id('cytd_e').charge=0
model.metabolites.get_by_id('dtdp4d6dm_c').charge=-2
model.metabolites.get_by_id('fgam_c').charge=-2
model.metabolites.get_by_id('cytd_p').charge=0
model.metabolites.get_by_id('dtdp_c').charge=-3
model.metabolites.get_by_id('dad_2_c').charge=0
model.metabolites.get_by_id('dtdpglu_c').charge=-2
model.metabolites.get_by_id('dtdprmn_c').charge=-2
model.metabolites.get_by_id('dad_5_c').charge=0
model.metabolites.get_by_id('dtmp_c').charge=-2
model.metabolites.get_by_id('fmn_c').charge=-2
model.metabolites.get_by_id('dadp_c').charge=-3
model.metabolites.get_by_id('dtmp_e').charge=-2
model.metabolites.get_by_id('damp_c').charge=-2
model.metabolites.get_by_id('for_c').charge=-1
model.metabolites.get_by_id('dtmp_p').charge=-2
model.metabolites.get_by_id('for_e').charge=-1
model.metabolites.get_by_id('damp_e').charge=-2
model.metabolites.get_by_id('dttp_c').charge=-4
model.metabolites.get_by_id('damp_p').charge=-2
model.metabolites.get_by_id('for_p').charge=-1
model.metabolites.get_by_id('dudp_c').charge=-3
model.metabolites.get_by_id('fpram_c').charge=-1
model.metabolites.get_by_id('dann_c').charge=1
model.metabolites.get_by_id('dump_c').charge=-2
model.metabolites.get_by_id('datp_c').charge=-4
model.metabolites.get_by_id('fprica_c').charge=-2
model.metabolites.get_by_id('dump_e').charge=-2
model.metabolites.get_by_id('frdp_c').charge=-3
model.metabolites.get_by_id('db4p_c').charge=-2
model.metabolites.get_by_id('dump_p').charge=-2
model.metabolites.get_by_id('fru_c').charge=0
model.metabolites.get_by_id('fru_e').charge=0
model.metabolites.get_by_id('dcaACP_c').charge=-1
model.metabolites.get_by_id('fru_p').charge=0
model.metabolites.get_by_id('dca_e').charge=-1
model.metabolites.get_by_id('dca_p').charge=-1
model.metabolites.get_by_id('dutp_c').charge=-4
model.metabolites.get_by_id('frulys_e').charge=1
model.metabolites.get_by_id('dxyl5p_c').charge=-2
model.metabolites.get_by_id('frulys_p').charge=1
model.metabolites.get_by_id('dcacoa_c').charge=-4
model.metabolites.get_by_id('dcamp_c').charge=-4
model.metabolites.get_by_id('e4p_c').charge=-2
model.metabolites.get_by_id('fruur_e').charge=-1
model.metabolites.get_by_id('dcdp_c').charge=-3
model.metabolites.get_by_id('dcmp_c').charge=-2
model.metabolites.get_by_id('fruur_p').charge=-1
model.metabolites.get_by_id('dcmp_e').charge=-2
model.metabolites.get_by_id('dcmp_p').charge=-2
model.metabolites.get_by_id('dctp_c').charge=-4
model.metabolites.get_by_id('dcyt_c').charge=0
model.metabolites.get_by_id('fum_c').charge=-2
model.metabolites.get_by_id('fum_e').charge=-2
model.metabolites.get_by_id('fum_p').charge=-2
model.metabolites.get_by_id('eig3p_c').charge=-2
model.metabolites.get_by_id('g1p_c').charge=-2
model.metabolites.get_by_id('enlipa_e').charge=-6
model.metabolites.get_by_id('ddcaACP_c').charge=-1
model.metabolites.get_by_id('g1p_e').charge=-2
model.metabolites.get_by_id('enlipa_p').charge=-6
model.metabolites.get_by_id('g1p_p').charge=-2
model.metabolites.get_by_id('g3p_c').charge=-2
model.metabolites.get_by_id('ddca_e').charge=-1
model.metabolites.get_by_id('ddca_p').charge=-1
model.metabolites.get_by_id('g3pc_e').charge=0
model.metabolites.get_by_id('ddcacoa_c').charge=-4
model.metabolites.get_by_id('dgdp_c').charge=-3
model.metabolites.get_by_id('g3pc_p').charge=0
model.metabolites.get_by_id('dgmp_c').charge=-2
model.metabolites.get_by_id('etha_e').charge=1
model.metabolites.get_by_id('dgmp_e').charge=-2
model.metabolites.get_by_id('g3pe_e').charge=0
model.metabolites.get_by_id('g3pe_p').charge=0
model.metabolites.get_by_id('etha_p').charge=1
model.metabolites.get_by_id('dgmp_p').charge=-2
model.metabolites.get_by_id('dgsn_c').charge=0
model.metabolites.get_by_id('g3pg_e').charge=-1
model.metabolites.get_by_id('ethso3_e').charge=-1
model.metabolites.get_by_id('g3pg_p').charge=-1
model.metabolites.get_by_id('dgsn_e').charge=0
model.metabolites.get_by_id('ethso3_p').charge=-1
model.metabolites.get_by_id('dgsn_p').charge=0
model.metabolites.get_by_id('dgtp_c').charge=-4
model.metabolites.get_by_id('etoh_c').charge=0
model.metabolites.get_by_id('g3pi_e').charge=-1
model.metabolites.get_by_id('g3pi_p').charge=-1
model.metabolites.get_by_id('etoh_e').charge=0
model.metabolites.get_by_id('dha_e').charge=0
model.metabolites.get_by_id('etoh_p').charge=0
model.metabolites.get_by_id('g3ps_e').charge=-1
model.metabolites.get_by_id('dha_p').charge=0
model.metabolites.get_by_id('dhap_c').charge=-2
model.metabolites.get_by_id('g3ps_p').charge=-1
model.metabolites.get_by_id('f6p_c').charge=-2
model.metabolites.get_by_id('g6p_c').charge=-2
model.metabolites.get_by_id('dhf_c').charge=-2
model.metabolites.get_by_id('f6p_e').charge=-2
model.metabolites.get_by_id('g6p_e').charge=-2
model.metabolites.get_by_id('f6p_p').charge=-2
model.metabolites.get_by_id('g6p_p').charge=-2
model.metabolites.get_by_id('dhmptp_c').charge=-4
model.metabolites.get_by_id('fad_c').charge=-2
model.metabolites.get_by_id('fadh2_c').charge=-2
model.metabolites.get_by_id('dhnpt_c').charge=0
model.metabolites.get_by_id('fald_c').charge=0
model.metabolites.get_by_id('gal1p_e').charge=-2
model.metabolites.get_by_id('fald_e').charge=0
model.metabolites.get_by_id('gal1p_p').charge=-2
model.metabolites.get_by_id('dhpmp_c').charge=-2
model.metabolites.get_by_id('fald_p').charge=0
model.metabolites.get_by_id('gal_bD_e').charge=0
model.metabolites.get_by_id('gal_bD_p').charge=0
model.metabolites.get_by_id('dhpt_c').charge=-1
model.metabolites.get_by_id('dhptd_c').charge=0
model.metabolites.get_by_id('gal_c').charge=0
model.metabolites.get_by_id('gal_e').charge=0
model.metabolites.get_by_id('didp_c').charge=-3
model.metabolites.get_by_id('fdp_c').charge=-4
model.metabolites.get_by_id('dimp_c').charge=-2
model.metabolites.get_by_id('gal_p').charge=0
model.metabolites.get_by_id('fe2_c').charge=2
model.metabolites.get_by_id('dimp_e').charge=-2
model.metabolites.get_by_id('fe2_e').charge=2
model.metabolites.get_by_id('dimp_p').charge=-2
model.metabolites.get_by_id('fe2_p').charge=2
model.metabolites.get_by_id('fe3_c').charge=3
model.metabolites.get_by_id('din_e').charge=0
model.metabolites.get_by_id('fe3_e').charge=3
model.metabolites.get_by_id('din_p').charge=0
model.metabolites.get_by_id('fe3_p').charge=3
model.metabolites.get_by_id('ditp_c').charge=-4
model.metabolites.get_by_id('galt_e').charge=0
model.metabolites.get_by_id('galt_p').charge=0
model.metabolites.get_by_id('galur_e').charge=-1
model.metabolites.get_by_id('dmlz_c').charge=0
model.metabolites.get_by_id('dmpp_c').charge=-3
model.metabolites.get_by_id('galur_p').charge=-1
model.metabolites.get_by_id('dms_e').charge=0
model.metabolites.get_by_id('gam1p_c').charge=-1
model.metabolites.get_by_id('gam6p_c').charge=-1
model.metabolites.get_by_id('gam6p_e').charge=-1
model.metabolites.get_by_id('dms_p').charge=0
model.metabolites.get_by_id('gam6p_p').charge=-1
model.metabolites.get_by_id('gam_e').charge=1
model.metabolites.get_by_id('dmso_e').charge=0
model.metabolites.get_by_id('dmso_p').charge=0
model.metabolites.get_by_id('gam_p').charge=1
model.metabolites.get_by_id('gar_c').charge=-1
model.metabolites.get_by_id('dnad_c').charge=-2
model.metabolites.get_by_id('dopa_e').charge=1
model.metabolites.get_by_id('gbbtn_e').charge=0
model.metabolites.get_by_id('gbbtn_p').charge=0
model.metabolites.get_by_id('gcald_c').charge=0
model.metabolites.get_by_id('gdp_e').charge=-3
model.metabolites.get_by_id('gdp_p').charge=-3
model.metabolites.get_by_id('gdp_c').charge=-3
model.metabolites.get_by_id('gdpmann_c').charge=-2
model.metabolites.get_by_id('h2_c').charge=0
model.metabolites.get_by_id('h2_e').charge=0
model.metabolites.get_by_id('h2_p').charge=0
model.metabolites.get_by_id('gdptp_c').charge=-7
model.metabolites.get_by_id('h2mb4p_c').charge=-3
model.metabolites.get_by_id('k_e').charge=1
model.metabolites.get_by_id('h2o2_c').charge=0
model.metabolites.get_by_id('h2o2_e').charge=0
model.metabolites.get_by_id('gg4abut_c').charge=-1
model.metabolites.get_by_id('k_p').charge=1
model.metabolites.get_by_id('ggbutal_c').charge=0
model.metabolites.get_by_id('h2o2_p').charge=0
model.metabolites.get_by_id('h2o_c').charge=0
model.metabolites.get_by_id('kdo2lipid4L_c').charge=-6
model.metabolites.get_by_id('h2o_e').charge=0
model.metabolites.get_by_id('ggptrc_c').charge=1
model.metabolites.get_by_id('kdo2lipid4_c').charge=-6
model.metabolites.get_by_id('h2o_p').charge=0
model.metabolites.get_by_id('h2s_c').charge=0
model.metabolites.get_by_id('h2s_e').charge=0
model.metabolites.get_by_id('h2s_p').charge=0
model.metabolites.get_by_id('h_c').charge=1
model.metabolites.get_by_id('kdo8p_c').charge=-3
model.metabolites.get_by_id('glcn_c').charge=-1
model.metabolites.get_by_id('h_e').charge=1
model.metabolites.get_by_id('kdo_c').charge=-1
model.metabolites.get_by_id('h_p').charge=1
model.metabolites.get_by_id('glcn_e').charge=-1
model.metabolites.get_by_id('glcn_p').charge=-1
model.metabolites.get_by_id('kdolipid4_c').charge=-5
model.metabolites.get_by_id('hco3_c').charge=-1
model.metabolites.get_by_id('glcr_e').charge=-2
model.metabolites.get_by_id('glcr_p').charge=-2
model.metabolites.get_by_id('glcur1p_e').charge=-3
model.metabolites.get_by_id('glcur1p_p').charge=-3
model.metabolites.get_by_id('hdca_e').charge=-1
model.metabolites.get_by_id('glcur_e').charge=-1
model.metabolites.get_by_id('hdca_p').charge=-1
model.metabolites.get_by_id('glcur_p').charge=-1
model.metabolites.get_by_id('hdcea_e').charge=-1
model.metabolites.get_by_id('hdcea_p').charge=-1
model.metabolites.get_by_id('hdcoa_c').charge=-4
model.metabolites.get_by_id('glu1sa_c').charge=0
model.metabolites.get_by_id('glu5p_c').charge=-2
model.metabolites.get_by_id('hdeACP_c').charge=-1
model.metabolites.get_by_id('lcts_e').charge=0
model.metabolites.get_by_id('glu5sa_c').charge=0
model.metabolites.get_by_id('lcts_p').charge=0
model.metabolites.get_by_id('hexACP_c').charge=-1
model.metabolites.get_by_id('hg2_e').charge=2
model.metabolites.get_by_id('hg2_p').charge=2
model.metabolites.get_by_id('hhlipa_c').charge=-6
model.metabolites.get_by_id('glucys_c').charge=-1
model.metabolites.get_by_id('glutrna_c').charge=0
model.metabolites.get_by_id('glx_c').charge=-1
model.metabolites.get_by_id('lipa_c').charge=-6
model.metabolites.get_by_id('gly_c').charge=0
model.metabolites.get_by_id('hisp_c').charge=-1
model.metabolites.get_by_id('gly_e').charge=0
model.metabolites.get_by_id('gly_p').charge=0
model.metabolites.get_by_id('glyald_c').charge=0
model.metabolites.get_by_id('histd_c').charge=1
model.metabolites.get_by_id('glyald_e').charge=0
model.metabolites.get_by_id('lipa_e').charge=-6
model.metabolites.get_by_id('glyald_p').charge=0
model.metabolites.get_by_id('lipa_p').charge=-6
model.metabolites.get_by_id('hlipa_c').charge=-6
model.metabolites.get_by_id('glyb_c').charge=0
model.metabolites.get_by_id('lipidA_c').charge=-4
model.metabolites.get_by_id('glyb_e').charge=0
model.metabolites.get_by_id('hmbil_c').charge=-8
model.metabolites.get_by_id('lipidAds_c').charge=-2
model.metabolites.get_by_id('hmfurn_c').charge=0
model.metabolites.get_by_id('glyb_p').charge=0
model.metabolites.get_by_id('lipidX_c').charge=-2
model.metabolites.get_by_id('glyc2p_e').charge=-2
model.metabolites.get_by_id('hmgth_c').charge=-1
model.metabolites.get_by_id('glyc2p_p').charge=-2
model.metabolites.get_by_id('glyc3p_c').charge=-2
model.metabolites.get_by_id('glyc3p_e').charge=-2
model.metabolites.get_by_id('glyc3p_p').charge=-2
model.metabolites.get_by_id('hpyr_c').charge=-1
model.metabolites.get_by_id('glyc_c').charge=0
model.metabolites.get_by_id('glyc_e').charge=0
model.metabolites.get_by_id('hxa_e').charge=-1
model.metabolites.get_by_id('hxa_p').charge=-1
model.metabolites.get_by_id('glyc_p').charge=0
model.metabolites.get_by_id('malACP_c').charge=-2
model.metabolites.get_by_id('glyclt_c').charge=-1
model.metabolites.get_by_id('glyclt_e').charge=-1
model.metabolites.get_by_id('hxan_c').charge=0
model.metabolites.get_by_id('hxan_e').charge=0
model.metabolites.get_by_id('hxan_p').charge=0
model.metabolites.get_by_id('glyclt_p').charge=-1
model.metabolites.get_by_id('hxcoa_c').charge=-4
model.metabolites.get_by_id('iasp_c').charge=-2
model.metabolites.get_by_id('gmhep17bp_c').charge=-4
model.metabolites.get_by_id('gmhep1p_c').charge=-2
model.metabolites.get_by_id('icit_c').charge=-3
model.metabolites.get_by_id('icolipa_c').charge=-11
model.metabolites.get_by_id('gmhep7p_c').charge=-2
model.metabolites.get_by_id('gmp_c').charge=-2
model.metabolites.get_by_id('gmp_e').charge=-2
model.metabolites.get_by_id('malcoa_c').charge=-5
model.metabolites.get_by_id('gmp_p').charge=-2
model.metabolites.get_by_id('malt6p_c').charge=-2
model.metabolites.get_by_id('idp_c').charge=-3
model.metabolites.get_by_id('grdp_c').charge=-3
model.metabolites.get_by_id('malt_c').charge=0
model.metabolites.get_by_id('grxox_c').charge=0
model.metabolites.get_by_id('malt_e').charge=0
model.metabolites.get_by_id('grxrd_c').charge=0
model.metabolites.get_by_id('malt_p').charge=0
model.metabolites.get_by_id('gsn_c').charge=0
model.metabolites.get_by_id('imacp_c').charge=-2
model.metabolites.get_by_id('gsn_e').charge=0
model.metabolites.get_by_id('imp_c').charge=-2
model.metabolites.get_by_id('imp_e').charge=-2
model.metabolites.get_by_id('gsn_p').charge=0
model.metabolites.get_by_id('gthox_c').charge=-2
model.metabolites.get_by_id('imp_p').charge=-2
model.metabolites.get_by_id('indole_c').charge=0
model.metabolites.get_by_id('gthox_e').charge=-2
model.metabolites.get_by_id('gthox_p').charge=-2
model.metabolites.get_by_id('indole_e').charge=0
model.metabolites.get_by_id('indole_p').charge=0
model.metabolites.get_by_id('gthrd_c').charge=-1
model.metabolites.get_by_id('gthrd_e').charge=-1
model.metabolites.get_by_id('inost_c').charge=0
model.metabolites.get_by_id('inost_e').charge=0
model.metabolites.get_by_id('gthrd_p').charge=-1
model.metabolites.get_by_id('gtp_c').charge=-4
model.metabolites.get_by_id('inost_p').charge=0
model.metabolites.get_by_id('ins_c').charge=0
model.metabolites.get_by_id('gtp_e').charge=-4
model.metabolites.get_by_id('gtp_p').charge=-4
model.metabolites.get_by_id('ipdp_c').charge=-3
model.metabolites.get_by_id('gua_c').charge=0
model.metabolites.get_by_id('man1p_c').charge=-2
model.metabolites.get_by_id('gua_e').charge=0
model.metabolites.get_by_id('isetac_e').charge=-1
model.metabolites.get_by_id('man6p_c').charge=-2
model.metabolites.get_by_id('isetac_p').charge=-1
model.metabolites.get_by_id('gua_p').charge=0
model.metabolites.get_by_id('man6p_e').charge=-2
model.metabolites.get_by_id('itp_c').charge=-4
model.metabolites.get_by_id('man6p_p').charge=-2
model.metabolites.get_by_id('k_c').charge=1
model.metabolites.get_by_id('man_e').charge=0
model.metabolites.get_by_id('o2_p').charge=0
model.metabolites.get_by_id('man_p').charge=0
model.metabolites.get_by_id('o2s_c').charge=-1
model.metabolites.get_by_id('manglyc_e').charge=-1
model.metabolites.get_by_id('o2s_e').charge=-1
model.metabolites.get_by_id('manglyc_p').charge=-1
model.metabolites.get_by_id('mg2_c').charge=2
model.metabolites.get_by_id('o2s_p').charge=-1
model.metabolites.get_by_id('oaa_c').charge=-2
model.metabolites.get_by_id('mg2_e').charge=2
model.metabolites.get_by_id('melib_e').charge=0
model.metabolites.get_by_id('mg2_p').charge=2
model.metabolites.get_by_id('micit_c').charge=-3
model.metabolites.get_by_id('ocACP_c').charge=-1
model.metabolites.get_by_id('melib_p').charge=0
model.metabolites.get_by_id('occoa_c').charge=-4
model.metabolites.get_by_id('ocdcaACP_c').charge=-1
model.metabolites.get_by_id('ocdca_e').charge=-1
model.metabolites.get_by_id('mlthf_c').charge=-2
model.metabolites.get_by_id('ocdca_p').charge=-1
model.metabolites.get_by_id('ocdcea_e').charge=-1
model.metabolites.get_by_id('mmet_e').charge=1
model.metabolites.get_by_id('ocdcea_p').charge=-1
model.metabolites.get_by_id('octa_e').charge=-1
model.metabolites.get_by_id('mmet_p').charge=1
model.metabolites.get_by_id('mn2_c').charge=2
model.metabolites.get_by_id('octa_p').charge=-1
model.metabolites.get_by_id('methf_c').charge=-1
model.metabolites.get_by_id('octdp_c').charge=-3
model.metabolites.get_by_id('mn2_e').charge=2
model.metabolites.get_by_id('mn2_p').charge=2
model.metabolites.get_by_id('octeACP_c').charge=-1
model.metabolites.get_by_id('pgp141_p').charge=-3
model.metabolites.get_by_id('mnl_e').charge=0
model.metabolites.get_by_id('mnl_p').charge=0
model.metabolites.get_by_id('mobd_c').charge=-2
model.metabolites.get_by_id('odecoa_c').charge=-4
model.metabolites.get_by_id('ohpb_c').charge=-3
model.metabolites.get_by_id('pgp160_c').charge=-3
model.metabolites.get_by_id('orn_c').charge=1
model.metabolites.get_by_id('mobd_e').charge=-2
model.metabolites.get_by_id('pgp160_p').charge=-3
model.metabolites.get_by_id('mobd_p').charge=-2
model.metabolites.get_by_id('orn_e').charge=1
model.metabolites.get_by_id('pgp161_c').charge=-3
model.metabolites.get_by_id('orn_p').charge=1
model.metabolites.get_by_id('pgp161_p').charge=-3
model.metabolites.get_by_id('orot5p_c').charge=-3
model.metabolites.get_by_id('pgp180_c').charge=-3
model.metabolites.get_by_id('orot_c').charge=-1
model.metabolites.get_by_id('orot_e').charge=-1
model.metabolites.get_by_id('mso3_e').charge=-1
model.metabolites.get_by_id('pgp180_p').charge=-3
model.metabolites.get_by_id('mso3_p').charge=-1
model.metabolites.get_by_id('orot_p').charge=-1
model.metabolites.get_by_id('pgp181_c').charge=-3
model.metabolites.get_by_id('mthgxl_c').charge=0
model.metabolites.get_by_id('pgp181_p').charge=-3
model.metabolites.get_by_id('murein3p3p_p').charge=-4
model.metabolites.get_by_id('pa120_c').charge=-2
model.metabolites.get_by_id('pa120_p').charge=-2
model.metabolites.get_by_id('pa140_c').charge=-2
model.metabolites.get_by_id('murein4p3p_p').charge=-4
model.metabolites.get_by_id('murein4p4p_p').charge=-4
model.metabolites.get_by_id('pa140_p').charge=-2
model.metabolites.get_by_id('pa141_c').charge=-2
model.metabolites.get_by_id('pheme_c').charge=-2
model.metabolites.get_by_id('pa141_p').charge=-2
model.metabolites.get_by_id('murein4px4p4p_p').charge=-6
model.metabolites.get_by_id('pheme_e').charge=-2
model.metabolites.get_by_id('murein4px4p_p').charge=-4
model.metabolites.get_by_id('pa160_c').charge=-2
model.metabolites.get_by_id('pa160_p').charge=-2
model.metabolites.get_by_id('murein4px4px4p_p').charge=-6
model.metabolites.get_by_id('pheme_p').charge=-2
model.metabolites.get_by_id('murein5p3p_p').charge=-4
model.metabolites.get_by_id('pa161_c').charge=-2
model.metabolites.get_by_id('pa161_p').charge=-2
model.metabolites.get_by_id('pa180_c').charge=-2
model.metabolites.get_by_id('murein5p4p_p').charge=-4
model.metabolites.get_by_id('murein5p5p5p_p').charge=-6
model.metabolites.get_by_id('pa180_p').charge=-2
model.metabolites.get_by_id('murein5p5p_p').charge=-4
model.metabolites.get_by_id('pa181_c').charge=-2
model.metabolites.get_by_id('phom_c').charge=-2
model.metabolites.get_by_id('pa181_p').charge=-2
model.metabolites.get_by_id('murein5px3p_p').charge=-4
model.metabolites.get_by_id('murein5px4p_p').charge=-4
model.metabolites.get_by_id('phpyr_c').charge=-1
model.metabolites.get_by_id('murein5px4px4p_p').charge=-6
model.metabolites.get_by_id('phthr_c').charge=-2
model.metabolites.get_by_id('myrsACP_c').charge=-1
model.metabolites.get_by_id('n2o_c').charge=0
model.metabolites.get_by_id('pacald_e').charge=0
model.metabolites.get_by_id('pi_c').charge=-2
model.metabolites.get_by_id('pacald_p').charge=0
model.metabolites.get_by_id('n2o_e').charge=0
model.metabolites.get_by_id('pi_e').charge=-2
model.metabolites.get_by_id('n2o_p').charge=0
model.metabolites.get_by_id('palmACP_c').charge=-1
model.metabolites.get_by_id('pi_p').charge=-2
model.metabolites.get_by_id('pan4p_c').charge=-2
model.metabolites.get_by_id('pmcoa_c').charge=-5
model.metabolites.get_by_id('na1_c').charge=1
model.metabolites.get_by_id('pap_c').charge=-4
model.metabolites.get_by_id('pmtcoa_c').charge=-4
model.metabolites.get_by_id('paps_c').charge=-4
model.metabolites.get_by_id('na1_e').charge=1
model.metabolites.get_by_id('na1_p').charge=1
model.metabolites.get_by_id('pdx5p_c').charge=-2
model.metabolites.get_by_id('pe120_c').charge=0
model.metabolites.get_by_id('nac_c').charge=-1
model.metabolites.get_by_id('nac_e').charge=-1
model.metabolites.get_by_id('pe120_p').charge=0
model.metabolites.get_by_id('ppa_c').charge=-1
model.metabolites.get_by_id('pe140_c').charge=0
model.metabolites.get_by_id('nac_p').charge=-1
model.metabolites.get_by_id('ppa_e').charge=-1
model.metabolites.get_by_id('nad_c').charge=-1
model.metabolites.get_by_id('pe140_p').charge=0
model.metabolites.get_by_id('ppa_p').charge=-1
model.metabolites.get_by_id('pe141_c').charge=0
model.metabolites.get_by_id('nadh_c').charge=-2
model.metabolites.get_by_id('ppal_c').charge=0
model.metabolites.get_by_id('nadp_c').charge=-3
model.metabolites.get_by_id('nadph_c').charge=-4
model.metabolites.get_by_id('pe141_p').charge=0
model.metabolites.get_by_id('pe160_c').charge=0
model.metabolites.get_by_id('ppal_e').charge=0
model.metabolites.get_by_id('pe160_p').charge=0
model.metabolites.get_by_id('ncam_c').charge=0
model.metabolites.get_by_id('ppal_p').charge=0
model.metabolites.get_by_id('nh4_c').charge=1
model.metabolites.get_by_id('ppap_c').charge=-2
model.metabolites.get_by_id('nh4_e').charge=1
model.metabolites.get_by_id('pe161_c').charge=0
model.metabolites.get_by_id('ppbng_c').charge=-1
model.metabolites.get_by_id('pe161_p').charge=0
model.metabolites.get_by_id('pe180_c').charge=0
model.metabolites.get_by_id('nh4_p').charge=1
model.metabolites.get_by_id('ppcoa_c').charge=-4
model.metabolites.get_by_id('ni2_c').charge=2
model.metabolites.get_by_id('pe180_p').charge=0
model.metabolites.get_by_id('ppgpp_c').charge=-6
model.metabolites.get_by_id('pe181_c').charge=0
model.metabolites.get_by_id('ni2_e').charge=2
model.metabolites.get_by_id('pphn_c').charge=-2
model.metabolites.get_by_id('ni2_p').charge=2
model.metabolites.get_by_id('pe181_p').charge=0
model.metabolites.get_by_id('nicrnt_c').charge=-2
model.metabolites.get_by_id('ppi_c').charge=-3
model.metabolites.get_by_id('peamn_e').charge=1
model.metabolites.get_by_id('ppp9_c').charge=-2
model.metabolites.get_by_id('peamn_p').charge=1
model.metabolites.get_by_id('nmn_c').charge=-1
model.metabolites.get_by_id('nmn_e').charge=-1
model.metabolites.get_by_id('nmn_p').charge=-1
model.metabolites.get_by_id('pep_c').charge=-3
model.metabolites.get_by_id('pg120_c').charge=-1
model.metabolites.get_by_id('pppg9_c').charge=-2
model.metabolites.get_by_id('pg120_p').charge=-1
model.metabolites.get_by_id('pppi_c').charge=-4
model.metabolites.get_by_id('no2_e').charge=-1
model.metabolites.get_by_id('pg140_c').charge=-1
model.metabolites.get_by_id('pg140_p').charge=-1
model.metabolites.get_by_id('pg141_c').charge=-1
model.metabolites.get_by_id('no2_p').charge=-1
model.metabolites.get_by_id('pppn_e').charge=-1
model.metabolites.get_by_id('pg141_p').charge=-1
model.metabolites.get_by_id('pppn_p').charge=-1
model.metabolites.get_by_id('pg160_c').charge=-1
model.metabolites.get_by_id('no3_e').charge=-1
model.metabolites.get_by_id('ppt_e').charge=-2
model.metabolites.get_by_id('no3_p').charge=-1
model.metabolites.get_by_id('pg160_p').charge=-1
model.metabolites.get_by_id('ppt_p').charge=-2
model.metabolites.get_by_id('pg161_c').charge=-1
model.metabolites.get_by_id('no_c').charge=0
model.metabolites.get_by_id('pram_c').charge=-1
model.metabolites.get_by_id('no_e').charge=0
model.metabolites.get_by_id('pg161_p').charge=-1
model.metabolites.get_by_id('pran_c').charge=-3
model.metabolites.get_by_id('pg180_c').charge=-1
model.metabolites.get_by_id('pg180_p').charge=-1
model.metabolites.get_by_id('no_p').charge=0
model.metabolites.get_by_id('prbamp_c').charge=-4
model.metabolites.get_by_id('prbatp_c').charge=-6
model.metabolites.get_by_id('pg181_c').charge=-1
model.metabolites.get_by_id('pg181_p').charge=-1
model.metabolites.get_by_id('prfp_c').charge=-4
model.metabolites.get_by_id('pgp120_c').charge=-3
model.metabolites.get_by_id('prlp_c').charge=-4
model.metabolites.get_by_id('pgp120_p').charge=-3
model.metabolites.get_by_id('pro__L_c').charge=0
model.metabolites.get_by_id('pgp140_c').charge=-3
model.metabolites.get_by_id('pgp140_p').charge=-3
model.metabolites.get_by_id('pgp141_c').charge=-3
model.metabolites.get_by_id('progly_c').charge=0
model.metabolites.get_by_id('o2_c').charge=0
model.metabolites.get_by_id('progly_e').charge=0
model.metabolites.get_by_id('o2_e').charge=0
model.metabolites.get_by_id('r1p_c').charge=-2
model.metabolites.get_by_id('progly_p').charge=0
model.metabolites.get_by_id('r5p_c').charge=-2
model.metabolites.get_by_id('r5p_e').charge=-2
model.metabolites.get_by_id('prpp_c').charge=-5
model.metabolites.get_by_id('r5p_p').charge=-2
model.metabolites.get_by_id('ps120_c').charge=-1
model.metabolites.get_by_id('tcynt_p').charge=-1
model.metabolites.get_by_id('ps140_c').charge=-1
model.metabolites.get_by_id('rbflvrd_c').charge=0
model.metabolites.get_by_id('ps141_c').charge=-1
model.metabolites.get_by_id('tdcoa_c').charge=-4
model.metabolites.get_by_id('ps160_c').charge=-1
model.metabolites.get_by_id('tddec2eACP_c').charge=-1
model.metabolites.get_by_id('rhcys_c').charge=0
model.metabolites.get_by_id('ps161_c').charge=-1
model.metabolites.get_by_id('tdeACP_c').charge=-1
model.metabolites.get_by_id('ps180_c').charge=-1
model.metabolites.get_by_id('tdec2eACP_c').charge=-1
model.metabolites.get_by_id('ps181_c').charge=-1
model.metabolites.get_by_id('tdecoa_c').charge=-4
model.metabolites.get_by_id('thdp_c').charge=-2
model.metabolites.get_by_id('ribflv_c').charge=0
model.metabolites.get_by_id('psclys_e').charge=1
model.metabolites.get_by_id('thex2eACP_c').charge=-1
model.metabolites.get_by_id('psclys_p').charge=1
model.metabolites.get_by_id('thf_c').charge=-2
model.metabolites.get_by_id('thm_c').charge=1
model.metabolites.get_by_id('rmn_e').charge=0
model.metabolites.get_by_id('rmn_p').charge=0
model.metabolites.get_by_id('thm_e').charge=1
model.metabolites.get_by_id('thm_p').charge=1
model.metabolites.get_by_id('thmmp_c').charge=-1
model.metabolites.get_by_id('thmpp_c').charge=-2
model.metabolites.get_by_id('s7p_c').charge=-2
model.metabolites.get_by_id('ptrc_c').charge=2
model.metabolites.get_by_id('ptrc_e').charge=2
model.metabolites.get_by_id('ptrc_p').charge=2
model.metabolites.get_by_id('pyam5p_c').charge=-1
model.metabolites.get_by_id('pydam_c').charge=1
model.metabolites.get_by_id('thrp_e').charge=-2
model.metabolites.get_by_id('pydx5p_c').charge=-2
model.metabolites.get_by_id('thrp_p').charge=-2
model.metabolites.get_by_id('pydx_c').charge=0
model.metabolites.get_by_id('pydxn_c').charge=0
model.metabolites.get_by_id('pyr_c').charge=-1
model.metabolites.get_by_id('pyr_e').charge=-1
model.metabolites.get_by_id('thym_e').charge=0
model.metabolites.get_by_id('thym_p').charge=0
model.metabolites.get_by_id('pyr_p').charge=-1
model.metabolites.get_by_id('thymd_c').charge=0
model.metabolites.get_by_id('q8_c').charge=0
model.metabolites.get_by_id('thymd_e').charge=0
model.metabolites.get_by_id('q8h2_c').charge=0
model.metabolites.get_by_id('thymd_p').charge=0
model.metabolites.get_by_id('quln_c').charge=-2
model.metabolites.get_by_id('tma_e').charge=1
model.metabolites.get_by_id('tma_p').charge=1
model.metabolites.get_by_id('tmao_e').charge=0
model.metabolites.get_by_id('sheme_c').charge=-8
model.metabolites.get_by_id('skm5p_c').charge=-3
model.metabolites.get_by_id('tmao_p').charge=0
model.metabolites.get_by_id('uaagmda_c').charge=-4
model.metabolites.get_by_id('tmrs2eACP_c').charge=-1
model.metabolites.get_by_id('skm_c').charge=-1
model.metabolites.get_by_id('toct2eACP_c').charge=-1
model.metabolites.get_by_id('uaccg_c').charge=-3
model.metabolites.get_by_id('toctd2eACP_c').charge=-1
model.metabolites.get_by_id('skm_e').charge=-1
model.metabolites.get_by_id('uacgam_c').charge=-2
model.metabolites.get_by_id('skm_p').charge=-1
model.metabolites.get_by_id('sl26da_c').charge=-2
model.metabolites.get_by_id('tpalm2eACP_c').charge=-1
model.metabolites.get_by_id('uacgam_e').charge=-2
model.metabolites.get_by_id('trdox_c').charge=0
model.metabolites.get_by_id('sl2a6o_c').charge=-3
model.metabolites.get_by_id('uacgam_p').charge=-2
model.metabolites.get_by_id('so2_c').charge=0
model.metabolites.get_by_id('trdrd_c').charge=0
model.metabolites.get_by_id('so2_e').charge=0
model.metabolites.get_by_id('so2_p').charge=0
model.metabolites.get_by_id('uagmda_c').charge=-4
model.metabolites.get_by_id('tre_e').charge=0
model.metabolites.get_by_id('so3_c').charge=-2
model.metabolites.get_by_id('so3_e').charge=-2
model.metabolites.get_by_id('uama_c').charge=-3
model.metabolites.get_by_id('so3_p').charge=-2
model.metabolites.get_by_id('tre_p').charge=0
model.metabolites.get_by_id('uamag_c').charge=-4
model.metabolites.get_by_id('so4_c').charge=-2
model.metabolites.get_by_id('uamr_c').charge=-3
model.metabolites.get_by_id('so4_e').charge=-2
model.metabolites.get_by_id('udcpdp_c').charge=-3
model.metabolites.get_by_id('so4_p').charge=-2
model.metabolites.get_by_id('spmd_c').charge=3
model.metabolites.get_by_id('spmd_e').charge=3
model.metabolites.get_by_id('udcpdp_p').charge=-3
model.metabolites.get_by_id('spmd_p').charge=3
model.metabolites.get_by_id('udcpp_c').charge=-2
model.metabolites.get_by_id('stcoa_c').charge=-4
model.metabolites.get_by_id('trnaglu_c').charge=0
model.metabolites.get_by_id('udcpp_p').charge=-2
model.metabolites.get_by_id('suc6p_c').charge=-2
model.metabolites.get_by_id('succ_c').charge=-2
model.metabolites.get_by_id('succ_e').charge=-2
model.metabolites.get_by_id('udp_c').charge=-3
model.metabolites.get_by_id('succ_p').charge=-2
model.metabolites.get_by_id('udpacgal_e').charge=-2
model.metabolites.get_by_id('succoa_c').charge=-5
model.metabolites.get_by_id('udpacgal_p').charge=-2
model.metabolites.get_by_id('udpg_c').charge=-2
model.metabolites.get_by_id('suchms_c').charge=-1
model.metabolites.get_by_id('udpg_e').charge=-2
model.metabolites.get_by_id('udpg_p').charge=-2
model.metabolites.get_by_id('sucr_e').charge=0
model.metabolites.get_by_id('udpgal_c').charge=-2
model.metabolites.get_by_id('udpgal_e').charge=-2
model.metabolites.get_by_id('sucr_p').charge=0
model.metabolites.get_by_id('sucsal_c').charge=-1
model.metabolites.get_by_id('udpgal_p').charge=-2
model.metabolites.get_by_id('sulfac_e').charge=-2
model.metabolites.get_by_id('sulfac_p').charge=-2
model.metabolites.get_by_id('udpglcur_e').charge=-3
model.metabolites.get_by_id('t3c5ddeceACP_c').charge=-1
model.metabolites.get_by_id('udpglcur_p').charge=-3
model.metabolites.get_by_id('tsul_c').charge=-2
model.metabolites.get_by_id('t3c7mrseACP_c').charge=-1
model.metabolites.get_by_id('ugmd_c').charge=-4
model.metabolites.get_by_id('t3c9palmeACP_c').charge=-1
model.metabolites.get_by_id('tsul_e').charge=-2
model.metabolites.get_by_id('ugmda_c').charge=-4
model.metabolites.get_by_id('tsul_p').charge=-2
model.metabolites.get_by_id('um4p_c').charge=-4
model.metabolites.get_by_id('ump_c').charge=-2
model.metabolites.get_by_id('ttdca_e').charge=-1
model.metabolites.get_by_id('ttdca_p').charge=-1
model.metabolites.get_by_id('ump_e').charge=-2
model.metabolites.get_by_id('ump_p').charge=-2
model.metabolites.get_by_id('ttdcea_e').charge=-1
model.metabolites.get_by_id('unaga_c').charge=-2
model.metabolites.get_by_id('ttdcea_p').charge=-1
model.metabolites.get_by_id('taur_e').charge=0
model.metabolites.get_by_id('tungs_e').charge=-2
model.metabolites.get_by_id('taur_p').charge=0
model.metabolites.get_by_id('tungs_p').charge=-2
model.metabolites.get_by_id('uppg3_c').charge=-8
model.metabolites.get_by_id('tym_e').charge=1
model.metabolites.get_by_id('tcynt_e').charge=-1
model.metabolites.get_by_id('ura_c').charge=0
model.metabolites.get_by_id('tym_p').charge=1
model.metabolites.get_by_id('ura_e').charge=0
model.metabolites.get_by_id('ura_p').charge=0
model.metabolites.get_by_id('tyrp_e').charge=-2
model.metabolites.get_by_id('tyrp_p').charge=-2
model.metabolites.get_by_id('u23ga_c').charge=-2
model.metabolites.get_by_id('urea_c').charge=0
model.metabolites.get_by_id('u3aga_c').charge=-2
model.metabolites.get_by_id('urea_e').charge=0
model.metabolites.get_by_id('u3hga_c').charge=-1
model.metabolites.get_by_id('urea_p').charge=0
model.metabolites.get_by_id('uri_c').charge=0
model.metabolites.get_by_id('utp_c').charge=-4
model.metabolites.get_by_id('xan_c').charge=0
model.metabolites.get_by_id('xan_e').charge=0
model.metabolites.get_by_id('xan_p').charge=0
model.metabolites.get_by_id('xmp_c').charge=-2
model.metabolites.get_by_id('xmp_e').charge=-2
model.metabolites.get_by_id('xmp_p').charge=-2
model.metabolites.get_by_id('xtp_c').charge=-4
model.metabolites.get_by_id('xtsn_c').charge=0
model.metabolites.get_by_id('xtsn_e').charge=0
model.metabolites.get_by_id('xtsn_p').charge=0
model.metabolites.get_by_id('zn2_c').charge=2
model.metabolites.get_by_id('zn2_e').charge=2
model.metabolites.get_by_id('zn2_p').charge=2

In [148]:
#Define biomass reactions to exclude from imbalance check
biomass_reactions = set(['PLIPIDS', 'PLIPIDS_ESS', 'PROTS', 'PEPGLY','RNAS', 'DNAS' ])


In [149]:
#Define remaining imbalances
#Many of these are due to the presence of cofactors in the reaction equation. In favor of correct stoichiometries, these cofactors are removed. 
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv('imbalancesinitial.csv')
mass_charge

121 121 121


,Reactions,Charge,Masses
0,3HAD140,-1.0,{}
1,3OAS161a,2.0,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
2,ACLS,1.0,{}
3,ADPT,-1.0,{}
4,ADSS,-1.0,{}
...,...,...,...
116,CPH4S,1.0,"{'C': -7.0, 'H': -8.0, 'N': -5.0, 'O': -3.0}"
117,CDGS,,"{'H': 3.0, 'N': 1.0}"
118,CDGR,-1.0,{'H': -5.0}
119,CCGS,-1.0,"{'N': -1.0, 'O': 2.0}"


In [150]:
#ASNSYN
r = model.reactions.get_by_id('ASNSYN')
print(r.check_mass_balance())
r

{'charge': -3.0, 'H': 2.0, 'O': 2.0}


Reaction identifier,ASNSYN
Name,R_ASNSYN
Memory address,0x014878d38cd0
Stoichiometry,asp_L_c + atp_c + gln_L_c + h_c --> adp_c + asn_L_c + glu_L_c + h2o_c + pi_c L-Aspartate + ATP + L-Glutamine + H --> ADP + L-Asparagine + L-Glutamate + H2O + Phosphate
GPR,NGO1489 and NGO0663 and NGO0662 and NGO0660
Lower bound,0.0
Upper bound,1000.0


In [151]:
h_c = model.metabolites.get_by_id('h_c')
h2o_c = model.metabolites.get_by_id('h2o_c')

r.add_metabolites({
    h2o_c: -2.0,
    h_c: 2.0})

model.metabolites.asp_L_c.charge=-1
print(r)
print(r.check_mass_balance())

ASNSYN: asp_L_c + atp_c + gln_L_c + h2o_c --> adp_c + asn_L_c + glu_L_c + h_c + pi_c
{}


In [152]:
#GLCTR1
r1 = model.reactions.get_by_id('GLCTR1')
print(r1.check_mass_balance())
r1

{'charge': -11.0}


Reaction identifier,GLCTR1
Name,R_GLCTR1
Memory address,0x01487aae4130
Stoichiometry,galglcgnahhlipa_c + udpg_c --> h_c + icolipa_c + udp_c M_galglcgnahhlipa_c + UDPglucose --> H + inner-core-oligosaccharide-lipid-A + UDP
GPR,NGO1353
Lower bound,0.0
Upper bound,1000.0


In [153]:
#SIALT1
r2 = model.reactions.get_by_id('SIALT1')
print(r2.check_mass_balance())
r2

{'charge': -2.0}


Reaction identifier,SIALT1
Name,R_SIALT1
Memory address,0x01487abbd820
Stoichiometry,cmpacna_c + los_GC_c --> cmp_c + sialylated_los_GC_c CMP-N-acetyl neuraminate + los_GC_c --> CMP + sialylated_los_GC_c
GPR,NGO1081
Lower bound,0.0
Upper bound,1000.0


In [154]:
#For GLCTR1 - Formula for icolipa_c is incorrect. When glc is added to peagnahhlipa_c, the previous reaction did not add the oxygens. 
#The corresponding reactions in LOS synthesis all are also missing these 5 oxygens. This is corrected here.
#Used structure diagrams here PMID: 11496013 and here PMID: 30873172 to determine proper formulas. 

#The reaction GLCTR1 corresponds to a bigg ID for a slightly different reaction in E.coli LPS biosynthesis. 
#This was left in place, but may need to be changed at a later time to avoid confusion.

#Additionally for SIALT1 - LPS is sialylated, the NeuAC was not actually added to the structure for LPS.  
#This is corrected here. Note this sialylation is on the alpha chain of LOS only, which is more common. Not on the beta chain.


#Correct the icolipa formula and formulas for the alphachain components to reflect the new 4HEXG beta chain
r = model.reactions.get_by_id('GLCTR1')
model.metabolites.get_by_id('icolipa_c').formula= 'C144H252N3O71P2'   
model.metabolites.get_by_id('galicolipa_c').formula= 'C150H262N3O76P2'
model.metabolites.get_by_id('gnagalicolipa_c').formula= 'C158H276N4O81P2'
model.metabolites.get_by_id('los_GC_c').formula= 'C164H286N4O86P2'

model.metabolites.get_by_id('sialylated_los_GC_c').formula= 'C175H304N5O94P2'
model.metabolites.galglcgnahhlipa_c.charge=-5
model.metabolites.peagnahhlipa_c.charge=-10
model.metabolites.sialylated_los_GC_c.charge=-4
model.metabolites.los_GC_c.charge=-4
model.metabolites.galicolipa_c.charge=-5
model.metabolites.gnagalicolipa_c.charge=-4
model.metabolites.icolipa_c.charge=-5
model.metabolites.glcgnahhlipa_c.charge=-5
model.metabolites.gnahhlipa_c.charge=-5
model.metabolites.glcgnahhlipa_c.charge=-5

model.metabolites.gnahhlipa_c.charge=-5
print(r)
print(r.check_mass_balance())
print(r1)
print(r1.check_mass_balance())
print(r2)
print(r2.check_mass_balance())

GLCTR1: galglcgnahhlipa_c + udpg_c --> h_c + icolipa_c + udp_c
{}
GLCTR1: galglcgnahhlipa_c + udpg_c --> h_c + icolipa_c + udp_c
{}
SIALT1: cmpacna_c + los_GC_c --> cmp_c + sialylated_los_GC_c
{'charge': -2.0}


In [155]:
r=model.reactions.get_by_id('GALT3')
print(r)
print(r.check_mass_balance())

GALT3: glcgnahhlipa_c + udpg_c --> galglcgnahhlipa_c + h_c + udp_c
{}


In [156]:
#Check all the steps of LOS synthesis to ensure its balanced properly. 
#SIALT1
r = model.reactions.get_by_id('SIALT1')
print(r.check_mass_balance())
r

{'charge': -2.0}


Reaction identifier,SIALT1
Name,R_SIALT1
Memory address,0x01487abbd820
Stoichiometry,cmpacna_c + los_GC_c --> cmp_c + sialylated_los_GC_c CMP-N-acetyl neuraminate + los_GC_c --> CMP + sialylated_los_GC_c
GPR,NGO1081
Lower bound,0.0
Upper bound,1000.0


In [157]:
#GALT2
r = model.reactions.get_by_id('GALT2')
print(r.check_mass_balance())
r

{}


Reaction identifier,GALT2
Name,R_GALT2
Memory address,0x01487aadca00
Stoichiometry,gnagalicolipa_c + udpgal_c --> h_c + los_GC_c + udp_c glcNAc-galactosyl-inner-core-oligosaccharide-lipid-A + UDPgalactose --> H + los_GC_c + UDP
GPR,NGO2156
Lower bound,0.0
Upper bound,1000.0


In [158]:
#GNAT2
r = model.reactions.get_by_id('GNAT2')

print(r.check_mass_balance())
r

{}


Reaction identifier,GNAT2
Name,R_GNAT2
Memory address,0x01487aaf5f10
Stoichiometry,"galicolipa_c + uacgam_c --> gnagalicolipa_c + udp_c galactosyl-inner-core-oligosaccharide-lipid-A, also mature lps, immunotype 8 + UDP-N-acetyl-D-glucosamine --> glcNAc-galactosyl-inner-core-oligosaccharide-lipid-A + UDP"
GPR,NGO2158
Lower bound,0.0
Upper bound,1000.0


In [159]:
#GALT1
r = model.reactions.get_by_id('GALT1')
print(r.check_mass_balance())
r

{}


Reaction identifier,GALT1
Name,R_GALT1
Memory address,0x01487aadcd30
Stoichiometry,"icolipa_c + udpgal_c --> galicolipa_c + h_c + udp_c inner-core-oligosaccharide-lipid-A + UDPgalactose --> galactosyl-inner-core-oligosaccharide-lipid-A, also mature lps, immunotype 8 + H + UDP"
GPR,NGO2159
Lower bound,0.0
Upper bound,1000.0


In [160]:
#GALT3
r = model.reactions.get_by_id('GALT3')
print(r.check_mass_balance())
r

{}


Reaction identifier,GALT3
Name,R_GLCTR2
Memory address,0x01487ac26b50
Stoichiometry,glcgnahhlipa_c + udpg_c --> galglcgnahhlipa_c + h_c + udp_c M_glcgnahhlipa_c + UDPglucose --> M_galglcgnahhlipa_c + H + UDP
GPR,NGO2072
Lower bound,0.0
Upper bound,1000.0


In [161]:
#GLCTR2
r = model.reactions.get_by_id('GLCTR2')
print(r.check_mass_balance())
r

{}


Reaction identifier,GLCTR2
Name,R_GLCTR2
Memory address,0x01487ac26fd0
Stoichiometry,gnahhlipa_c + udpg_c --> glcgnahhlipa_c + h_c + udp_c glcNAc-heptosyl-heptosyl-kdo2-lipidA + UDPglucose --> M_glcgnahhlipa_c + H + UDP
GPR,NGO2072
Lower bound,0.0
Upper bound,1000.0


In [162]:
#GLCTR1
r = model.reactions.get_by_id('GLCTR1')
print(r.check_mass_balance())
r

{}


Reaction identifier,GLCTR1
Name,R_GLCTR1
Memory address,0x01487aae4130
Stoichiometry,galglcgnahhlipa_c + udpg_c --> h_c + icolipa_c + udp_c M_galglcgnahhlipa_c + UDPglucose --> H + inner-core-oligosaccharide-lipid-A + UDP
GPR,NGO1353
Lower bound,0.0
Upper bound,1000.0


In [163]:
#GNAT1
r = model.reactions.get_by_id('GNAT1')
print(r.check_mass_balance())
r

{}


Reaction identifier,GNAT1
Name,R_GNAT1
Memory address,0x01487aaf58e0
Stoichiometry,hhlipa_c + uacgam_c --> gnahhlipa_c + udp_c heptosyl-heptosyl-kdo2-lipidA + UDP-N-acetyl-D-glucosamine --> glcNAc-heptosyl-heptosyl-kdo2-lipidA + UDP
GPR,NGO1354
Lower bound,0.0
Upper bound,1000.0


In [164]:
#HEPT2
r = model.reactions.get_by_id('HEPT2')
print(r.check_mass_balance())
r

{}


Reaction identifier,HEPT2
Name,R_HEPT2
Memory address,0x01487aafde20
Stoichiometry,adphep_LD_c + hlipa_c --> adp_c + h_c + hhlipa_c ADP-L-glycero-D-manno-heptose + heptosyl-kdo2-lipidA --> ADP + H + heptosyl-heptosyl-kdo2-lipidA
GPR,NGO0987
Lower bound,0.0
Upper bound,1000.0


In [165]:
#HEPT1
r = model.reactions.get_by_id('HEPT1')
print(r.check_mass_balance())
r

{}


Reaction identifier,HEPT1
Name,R_HEPT1
Memory address,0x01487aafdf10
Stoichiometry,adphep_LD_c + lipa_c --> adp_c + h_c + hlipa_c ADP-L-glycero-D-manno-heptose + KDO-2-lipid-A --> ADP + H + heptosyl-kdo2-lipidA
GPR,NGO1934
Lower bound,0.0
Upper bound,1000.0


In [166]:
#All of LOS synthesis looks correct. Continue checking/ correcting other reactions. 

In [167]:
model.slim_optimize()

35.34588087861347

In [168]:
#MAc3
r = model.reactions.get_by_id('MAc3')
print(r.check_mass_balance())
r

{'charge': 4.0}


Reaction identifier,MAc3
Name,R_MAc3
Memory address,0x01487ab14b80
Stoichiometry,accoa_c + h2o_p + murein5p4p_p --> coa_c + murein5p4pOAc_p Acetyl-CoA + H2O + two-linked-disacharide-pentapeptide-and-tetrapeptide-murein-units-uncrosslinked-middle-of-chain --> Coenzyme-A + two-linked-disacharide-pentapeptide-and-tetrapeptide-murein-...
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,1000.0


In [169]:
# Both Gc and Nm acylate their peptidoglycan. PMCID: PMC1231103
print(model.metabolites.get_by_id('murein5p4p_p').formula)
print(model.metabolites.get_by_id('murein5p4pOAc_p').formula)


C77H119N15O41
C79H123N15O43


In [170]:
#In the formula for murein5p4pOAc, the oxygen that gets added during acylation was forgotten. This is added here. 
model.metabolites.get_by_id('murein5p4pOAc_p').formula= 'C79H123N15O43'  
model.metabolites.murein5p4pOAc_p.charge=-4

In [171]:
print(r.check_mass_balance())
r

{}


Reaction identifier,MAc3
Name,R_MAc3
Memory address,0x01487ab14b80
Stoichiometry,accoa_c + h2o_p + murein5p4p_p --> coa_c + murein5p4pOAc_p Acetyl-CoA + H2O + two-linked-disacharide-pentapeptide-and-tetrapeptide-murein-units-uncrosslinked-middle-of-chain --> Coenzyme-A + two-linked-disacharide-pentapeptide-and-tetrapeptide-murein-...
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,1000.0


In [172]:
#PHEMEFErel
r = model.reactions.get_by_id('PHEMEFErel')
print(r.check_mass_balance())
r

{'charge': 7.0}


Reaction identifier,PHEMEFErel
Name,R_PHEMEFErel
Memory address,0x01487ab62970
Stoichiometry,3.0 h_c + 6.0 nadph_c + 3.0 o2_c + pheme_c --> biliverd_c + co_c + fe2_c + 3.0 h2o_c + 6.0 nadp_c 3.0 H + 6.0 Nicotinamide-adenine-dinucleotide-phosphate--reduced + 3.0 O2 + Protoheme --> Biliverdin + Carbon monoxide + Fe2 + 3.0 H2O + 6.0 Nicotinamide-adenine-dinucleotide-phosphate
GPR,NGO1318
Lower bound,0.0
Upper bound,1000.0


In [173]:
#Product of a simplified, 3 step reaction that requires 2 NADPHs. PMCID: PMC111422. 
#This reaction is described in kegg as requiring 3 "electron donors" and 3"electron acceptors", which is likely the cause of the error. https://www.genome.jp/entry/R11816

print(r.check_mass_balance())
r
#Leaving charge imbalanced as its a simplified reaction

{'charge': 7.0}


Reaction identifier,PHEMEFErel
Name,R_PHEMEFErel
Memory address,0x01487ab62970
Stoichiometry,3.0 h_c + 6.0 nadph_c + 3.0 o2_c + pheme_c --> biliverd_c + co_c + fe2_c + 3.0 h2o_c + 6.0 nadp_c 3.0 H + 6.0 Nicotinamide-adenine-dinucleotide-phosphate--reduced + 3.0 O2 + Protoheme --> Biliverdin + Carbon monoxide + Fe2 + 3.0 H2O + 6.0 Nicotinamide-adenine-dinucleotide-phosphate
GPR,NGO1318
Lower bound,0.0
Upper bound,1000.0


In [174]:
#PMCOAS
r = model.reactions.get_by_id('PMCOAS')
print(r.check_mass_balance())
r
#Okay to leave unbalanced. THis reaction is a simplification of a multistep ACP dependent reaction that isn't well understood. 

{'charge': -1.0, 'C': 7.0, 'H': 9.0, 'O': 3.0}


Reaction identifier,PMCOAS
Name,R_PMCOAS
Memory address,0x01487ab6edf0
Stoichiometry,coa_c --> pmcoa_c Coenzyme-A --> Pimeloyl-CoA
GPR,NGO1481 and NGO1725
Lower bound,0.0
Upper bound,1000.0


In [175]:
#TFFE
r = model.reactions.get_by_id('TFFE')
print(r.check_mass_balance())
r


{'charge': 3.0}


Reaction identifier,TFFE
Name,R_TFFE
Memory address,0x01487abd3eb0
Stoichiometry,TfFe3_e --> Tf_e + fe3_p Transferrin --> Transferrin residue + Fe3
GPR,NGO1496 and NGO1495 and NGO1379 and NGO1378 and ( Blank or NGO1377 ) and NGO0217 and NGO0216 and...
Lower bound,0.0
Upper bound,1000.0


In [176]:
#Although transferrin holds two irons, Gc is only capable of stripping one off at a time. So transferrin is made to produce a single iron here. 
fe3_p =model.metabolites.get_by_id('fe3_p')

model.metabolites.TfFe3_e.charge=3

print(r.check_mass_balance())
r

{}


Reaction identifier,TFFE
Name,R_TFFE
Memory address,0x01487abd3eb0
Stoichiometry,TfFe3_e --> Tf_e + fe3_p Transferrin --> Transferrin residue + Fe3
GPR,NGO1496 and NGO1495 and NGO1379 and NGO1378 and ( Blank or NGO1377 ) and NGO0217 and NGO0216 and...
Lower bound,0.0
Upper bound,1000.0


In [177]:
#Add thioredoxin import reaction. This is required to show "synthesis" of a protein based component. 
#Integration of Biomass Formulations of Genome-Scale Metabolic Models with Experimental Data Reveals Universally Essential Cofactors in Prokaryotes
#Joana C Xavier, Kiran Raosaheb Patil, Isabel Rocha
#PMID: 27939572 PMCID: PMC5249239

trdrd_c = model.metabolites.get_by_id('trdrd_c')

EX_trdrd_c_ = cobra.Reaction('EX_trdrd_c_')
EX_trdrd_c_.gene_reaction_rule = ''
EX_trdrd_c_.lower_bound = 0.
EX_trdrd_c_.upper_bound = 1000.
EX_trdrd_c_.add_metabolites({
    trdrd_c: -1.0,
})

model.add_reactions([EX_trdrd_c_])


Ignoring reaction 'EX_trdrd_c_' since it already exists.


In [178]:
#TSULST
#synthesis of thioredoxin. Protein based component. 
r = model.reactions.get_by_id('TSULST')
print(r.check_mass_balance())
r


{}


Reaction identifier,TSULST
Name,R_TSULST
Memory address,0x01487abf0dc0
Stoichiometry,acser_c + trdrd_c + tsul_c <=> ac_c + cys_L_c + h_c + so3_c + trdox_c O-Acetyl-L-serine + Reduced-thioredoxin + Thiosulfate <=> Acetate + L-Cysteine + H + Sulfite + Oxidized-thioredoxin
GPR,NGO0340
Lower bound,-1000.0
Upper bound,1000.0


In [179]:
h_c =model.metabolites.get_by_id('h_c')

r.add_metabolites({
    h_c: 2,
})

print(r.check_mass_balance())
r

{'charge': 2.0, 'H': 2.0}


Reaction identifier,TSULST
Name,R_TSULST
Memory address,0x01487abf0dc0
Stoichiometry,acser_c + trdrd_c + tsul_c <=> ac_c + cys_L_c + 3.0 h_c + so3_c + trdox_c O-Acetyl-L-serine + Reduced-thioredoxin + Thiosulfate <=> Acetate + L-Cysteine + 3.0 H + Sulfite + Oxidized-thioredoxin
GPR,NGO0340
Lower bound,-1000.0
Upper bound,1000.0


In [180]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv('imbalancesremaining.csv')
mass_charge

106 106 106


,Reactions,Charge,Masses
0,3HAD140,-1.0,{}
1,3OAS161a,2.0,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
2,ACLS,1.0,{}
3,ADPT,-1.0,{}
4,AGPAT160a,1.0,{}
...,...,...,...
101,CPH4S,1.0,"{'C': -7.0, 'H': -8.0, 'N': -5.0, 'O': -3.0}"
102,CDGS,,"{'H': 3.0, 'N': 1.0}"
103,CDGR,-1.0,{'H': -5.0}
104,CCGS,-1.0,"{'N': -1.0, 'O': 2.0}"


#DDPGA
r = model.reactions.get_by_id('DDPGA')
print(r.check_mass_balance())
r

In [181]:
#P3H5CD
r = model.reactions.get_by_id('P3H5CD')
print(r.check_mass_balance())
r

{'charge': 1.0}


Reaction identifier,P3H5CD
Name,R_P3H5CD
Memory address,0x01487ab47a60
Stoichiometry,1p3h5c_c + 2.0 h2o_c + nadp_c <=> e4hglu_c + 2.0 h_c + nadph_c L-1-Pyrroline-3-hydroxy-5-carboxylate + 2.0 H2O + Nicotinamide-adenine-dinucleotide-phosphate <=> L-erythro-4-Hydroxyglutamate + 2.0 H + Nicotinamide-adenine-dinucleotide-phosphate--reduced
GPR,NGO08225
Lower bound,-1000.0
Upper bound,1000.0


In [182]:
#The protonation of e4hglu was changed when correcting HGTA. This results in having an extra h consumed in P3H5CD.
#This is corrected here. 
h_c =model.metabolites.get_by_id('h_c')
model.metabolites.get_by_id('1p3h5c_c').charge=0

print(r.check_mass_balance())
r

{'charge': 1.0}


Reaction identifier,P3H5CD
Name,R_P3H5CD
Memory address,0x01487ab47a60
Stoichiometry,1p3h5c_c + 2.0 h2o_c + nadp_c <=> e4hglu_c + 2.0 h_c + nadph_c L-1-Pyrroline-3-hydroxy-5-carboxylate + 2.0 H2O + Nicotinamide-adenine-dinucleotide-phosphate <=> L-erythro-4-Hydroxyglutamate + 2.0 H + Nicotinamide-adenine-dinucleotide-phosphate--reduced
GPR,NGO08225
Lower bound,-1000.0
Upper bound,1000.0


In [183]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv('imbalancesremaining.csv')
mass_charge

106 106 106


,Reactions,Charge,Masses
0,3HAD140,-1.0,{}
1,3OAS161a,2.0,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
2,ACLS,1.0,{}
3,ADPT,-1.0,{}
4,AGPAT160a,1.0,{}
...,...,...,...
101,CPH4S,1.0,"{'C': -7.0, 'H': -8.0, 'N': -5.0, 'O': -3.0}"
102,CDGS,,"{'H': 3.0, 'N': 1.0}"
103,CDGR,-1.0,{'H': -5.0}
104,CCGS,-1.0,"{'N': -1.0, 'O': 2.0}"


In [184]:
r=model.reactions.AGPAT160a
print(r.check_mass_balance())
model.metabolites.get_by_id('pa160a_c').charge=-2
print(r.check_mass_balance())
r

{'charge': 1.0}
{}


Reaction identifier,AGPAT160a
Name,R_AGPAT160a
Memory address,0x014878d37b80
Stoichiometry,1hdecg3p_c + ddcaACP_c --> ACP_c + pa160a_c 1-hexadecanoyl-sn-glycerol-3-phosphate + Dodecanoyl-ACP-n-C120ACP --> acyl-carrier-protein + 1-hexadecanoyl-2-dodecanoyl-sn-glycerol-3-phosphate
GPR,NGO2069 or NGO0611
Lower bound,0.0
Upper bound,1000.0


In [185]:
model.metabolites.get_by_id('pa161_c').charge=-2
model.metabolites.get_by_id('pa181_c').charge=-2

In [186]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv('imbalancesremaining.csv')
mass_charge

106 106 106


,Reactions,Charge,Masses
0,3HAD140,-1.0,{}
1,3OAS161a,2.0,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
2,ACLS,1.0,{}
3,ADPT,-1.0,{}
4,AGPAT161a,-2.0,{}
...,...,...,...
101,CPH4S,1.0,"{'C': -7.0, 'H': -8.0, 'N': -5.0, 'O': -3.0}"
102,CDGS,,"{'H': 3.0, 'N': 1.0}"
103,CDGR,-1.0,{'H': -5.0}
104,CCGS,-1.0,"{'N': -1.0, 'O': 2.0}"


In [187]:
r=model.reactions.ACLS
model.metabolites.alac_S_c.charge=-1
print(r.check_mass_balance())
r

{}


Reaction identifier,ACLS
Name,R_ACLS
Memory address,0x014878d1ff10
Stoichiometry,h_c + 2.0 pyr_c --> alac_S_c + co2_c H + 2.0 Pyruvate --> S-2-Acetolactate + CO2
GPR,NGO1236 and ( Blank or NGO1235 )
Lower bound,0.0
Upper bound,1000.0


In [188]:
r=model.reactions.ADPT
r.add_metabolites({
    h_c: 1.0,
})
model.metabolites.prpp_c.formula='C5H9O14P3'
print(r.check_mass_balance())
r

{'charge': -1.0, 'H': -1.0}


Reaction identifier,ADPT
Name,R_ADPT
Memory address,0x014878cf3700
Stoichiometry,ade_c + prpp_c <=> amp_c + ppi_c Adenine + 5-Phospho-alpha-D-ribose-1-diphosphate <=> AMP + Diphosphate
GPR,NGO1311
Lower bound,-1000.0
Upper bound,1000.0


In [189]:
r=model.reactions.AMPTASEAGl
model.metabolites.glu_D_c.charge=-1
print(r.check_mass_balance())
r

{}


Reaction identifier,AMPTASEAGl
Name,R_AMPTASEAGl
Memory address,0x014878d5ef70
Stoichiometry,LalaDglu_c + h2o_c --> ala_L_c + glu_D_c L-alanine-D-glutamate + H2O --> L-Alanine + D-Glutamate
GPR,NGO0158
Lower bound,0.0
Upper bound,1000.0


In [190]:
r=model.reactions.ANPRT
r.add_metabolites({
    h_c: 1.0,
})
print(r.check_mass_balance())
r

{}


Reaction identifier,ANPRT
Name,R_ANPRT
Memory address,0x014878d5cdf0
Stoichiometry,anth_c + prpp_c --> h_c + ppi_c + pran_c Anthranilate + 5-Phospho-alpha-D-ribose-1-diphosphate --> H + Diphosphate + N-5-Phospho-D-ribosyl-anthranilate
GPR,NGO1203
Lower bound,0.0
Upper bound,1000.0


In [191]:
r=model.reactions.ARGDC
model.metabolites.arg_L_c.charge=1
model.metabolites.arg_L_p.charge=1
model.metabolites.arg_L_e.charge=1
print(r.check_mass_balance())
r

{}


Reaction identifier,ARGDC
Name,R_ARGDC
Memory address,0x014878d3e940
Stoichiometry,arg_L_c + h_c --> agm_c + co2_c L-Arginine + H --> Agmatine + CO2
GPR,NGO1487
Lower bound,0.0
Upper bound,1000.0


In [192]:
r=model.reactions.ASPtex
model.metabolites.asp_L_c.charge=-1
model.metabolites.asp_L_p.charge=-1
model.metabolites.asp_L_e.charge=-1
model.metabolites.glu_L_c.charge=-1
model.metabolites.glu_L_p.charge=-1
model.metabolites.glu_L_e.charge=-1
print(r.check_mass_balance())
r

{}


Reaction identifier,ASPtex
Name,R_ASPtex
Memory address,0x014878cd0e80
Stoichiometry,asp_L_e <=> asp_L_p L-Aspartate <=> L-Aspartate
GPR,NGO1812
Lower bound,-1000.0
Upper bound,1000.0


In [193]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv('imbalancesremaining.csv')
mass_charge

91 91 91


,Reactions,Charge,Masses
0,3HAD140,-1.0,{}
1,3OAS161a,2.0,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
2,ADPT,-1.0,{'H': -1.0}
3,AGPAT161a,-2.0,{}
4,AGPAT181,-2.0,{}
...,...,...,...
86,CPH4S,1.0,"{'C': -7.0, 'H': -8.0, 'N': -5.0, 'O': -3.0}"
87,CDGS,,"{'H': 3.0, 'N': 1.0}"
88,CDGR,-1.0,{'H': -5.0}
89,CCGS,-1.0,"{'N': -1.0, 'O': 2.0}"


In [194]:
r=model.reactions.get_by_id('ARGt1pp')
model.metabolites.arg_L_p.charge=1
print(r.check_mass_balance())
r

{}


Reaction identifier,ARGt1pp
Name,R_ARGt1pp
Memory address,0x014878d62eb0
Stoichiometry,arg_L_p + h_p --> arg_L_c + h_c L-Arginine + H --> L-Arginine + H
GPR,NGO1290 or NGO1787
Lower bound,0.0
Upper bound,1000.0


In [195]:
#There is a descrepancy between bigg and model seed for the charge/formula of atp. Using the bigg charges here. 
#These requires a charge imbalance to remain. 
r=model.reactions.get_by_id('CTPS2')
model.metabolites.atp_c.charge=-4
model.metabolites.atp_c.formula='C10H12N5O13P3'
model.metabolites.adp_c.charge=-3
model.metabolites.adp_c.formula='C10H12N5O10P2'
model.metabolites.amp_c.charge=-2
model.metabolites.amp_c.formula='C10H12N5O7P'
model.metabolites.ctp_c.charge=-4
model.metabolites.ctp_c.formula='C9H12N3O14P3'
print(r.check_mass_balance())
r

{}


Reaction identifier,CTPS2
Name,R_CTPS2
Memory address,0x014878cea880
Stoichiometry,atp_c + gln_L_c + h2o_c + utp_c --> adp_c + ctp_c + glu_L_c + 2.0 h_c + pi_c ATP + L-Glutamine + H2O + UTP --> ADP + CTP + L-Glutamate + 2.0 H + Phosphate
GPR,NGO1212
Lower bound,0.0
Upper bound,1000.0


In [202]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv('imbalancesremaining.csv')
mass_charge

91 91 91


,Reactions,Charge,Masses
0,3HAD140,-1.0,{}
1,3OAS161a,2.0,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
2,ADPT,-1.0,{'H': -1.0}
3,AGPAT161a,-2.0,{}
4,AGPAT181,-2.0,{}
...,...,...,...
86,CPH4S,1.0,"{'C': -7.0, 'H': -8.0, 'N': -5.0, 'O': -3.0}"
87,CDGS,,"{'H': 3.0, 'N': 1.0}"
88,CDGR,-1.0,{'H': -5.0}
89,CCGS,-1.0,"{'N': -1.0, 'O': 2.0}"


#Bulk correct hydrogens
rxns=( 'METS','ORPT')

for rxn in rxns:
    r=model.reactions.get_by_id(rxn)
    r.add_metabolites({
        h_c: -1.0,
            })
    print(r.check_mass_balance())



In [205]:
#Bulk correct hydrogens
rxns=('GLUPRT','GUAPRT','HXPRT','NAMNPP','NNDPR','UPPRT','XPPT','XPRT')

for rxn in rxns:
    r=model.reactions.get_by_id(rxn)
    r.add_metabolites({
        h_c: 1.0,
           })
    print(r.check_mass_balance())  

{}
{}
{}
{}
{}
{}
{}
{}


In [206]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv('imbalancesremaining.csv')
mass_charge

83 83 83


,Reactions,Charge,Masses
0,3HAD140,-1.0,{}
1,3OAS161a,2.0,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
2,ADPT,-1.0,{'H': -1.0}
3,AGPAT161a,-2.0,{}
4,AGPAT181,-2.0,{}
...,...,...,...
78,CPH4S,1.0,"{'C': -7.0, 'H': -8.0, 'N': -5.0, 'O': -3.0}"
79,CDGS,,"{'H': 3.0, 'N': 1.0}"
80,CDGR,-1.0,{'H': -5.0}
81,CCGS,-1.0,"{'N': -1.0, 'O': 2.0}"


In [207]:
r=model.reactions.get_by_id('DAPDC')
model.metabolites.get_by_id('lys_L_c').charge=1
model.metabolites.get_by_id('lys_L_p').charge=1
model.metabolites.get_by_id('lys_L_e').charge=1
print(r.check_mass_balance())
r

{}


Reaction identifier,DAPDC
Name,R_DAPDC
Memory address,0x014879ad9df0
Stoichiometry,26dap_M_c + h_c --> co2_c + lys_L_c meso-2-6-Diaminoheptanedioate + H --> CO2 + L-Lysine
GPR,NGO2098
Lower bound,0.0
Upper bound,1000.0


In [208]:
r=model.reactions.get_by_id('GALUi')
model.metabolites.utp_c.formula='C9H11N2O15P3'
model.metabolites.utp_c.charge=-4
print(r.check_mass_balance())
r

{}


Reaction identifier,GALUi
Name,R_GALUi
Memory address,0x01487aadc820
Stoichiometry,g1p_c + h_c + utp_c --> ppi_c + udpg_c D-Glucose-1-phosphate + H + UTP --> Diphosphate + UDPglucose
GPR,NGO0220
Lower bound,0.0
Upper bound,1000.0


In [209]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv('imbalancesremaining.csv')
mass_charge

82 82 82


,Reactions,Charge,Masses
0,3HAD140,-1.0,{}
1,3OAS161a,2.0,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
2,ADPT,-1.0,{'H': -1.0}
3,AGPAT161a,-2.0,{}
4,AGPAT181,-2.0,{}
...,...,...,...
77,CPH4S,1.0,"{'C': -7.0, 'H': -8.0, 'N': -5.0, 'O': -3.0}"
78,CDGS,,"{'H': 3.0, 'N': 1.0}"
79,CDGR,-1.0,{'H': -5.0}
80,CCGS,-1.0,"{'N': -1.0, 'O': 2.0}"


In [210]:
r=model.reactions.get_by_id('ATPPRT')
r.add_metabolites({
    h_c: 1.0,
      })

print(r.check_mass_balance())  

{}


In [215]:
r=model.reactions.get_by_id('PGSA161a')
r.add_metabolites({
    h_c: -1.0,
      })

print(r.check_mass_balance())

{'charge': -2.0}


In [216]:
r=model.reactions.get_by_id('PLIPA1G161a')
r.add_metabolites({
    h_c: 1.0,
      })

print(r.check_mass_balance())

{'charge': 1.0}


In [218]:
r=model.reactions.get_by_id('PRPPS')
r.add_metabolites({
    h_c: -1.0,
      })

print(r.check_mass_balance())

{}


In [219]:
r=model.reactions.get_by_id('HGFE')

print(r.check_mass_balance())
r
#okay to leave imbalanced

{'charge': -8.0}


Reaction identifier,HGFE
Name,R_HGFE
Memory address,0x01487aafdd30
Stoichiometry,HgFe3_e --> Hg_e + 4.0 pheme_p Hemoglobin --> Hemoglobin residue + 4.0 Protoheme
GPR,NGO2110 and NGO2109
Lower bound,0.0
Upper bound,1000.0


In [220]:
r=model.reactions.get_by_id('L_LACD2')
model.metabolites.lac_L_c.charge=-1
model.metabolites.lac_L_e.charge=-1
model.metabolites.lac_L_p.charge=-1
print(r.check_mass_balance())
r


{}


Reaction identifier,L_LACD2
Name,R_L_LACD2
Memory address,0x01487ab0bdf0
Stoichiometry,lac_L_c + q8_c --> pyr_c + q8h2_c L-Lactate + Ubiquinone-8 --> Pyruvate + Ubiquinol-8
GPR,NGO0639 or ( NGO0904 and NGO0905 and NGO0906 )
Lower bound,0.0
Upper bound,1000.0


In [221]:
r=model.reactions.get_by_id('LDH_D')
model.metabolites.lac_D_c.charge=-1
model.metabolites.lac_D_e.charge=-1
model.metabolites.lac_D_p.charge=-1
print(r.check_mass_balance())
r

{}


Reaction identifier,LDH_D
Name,R_LDH_D
Memory address,0x01487ab11ee0
Stoichiometry,lac_D_c + nad_c <=> h_c + nadh_c + pyr_c D-Lactate + Nicotinamide-adenine-dinucleotide <=> H + Nicotinamide-adenine-dinucleotide--reduced + Pyruvate
GPR,NGO1336
Lower bound,-1000.0
Upper bound,1000.0


In [222]:
r=model.reactions.get_by_id('LGTHL')
model.metabolites.lgt_S_c.charge=-1
print(r.check_mass_balance())
r


{}


Reaction identifier,LGTHL
Name,R_LGTHL
Memory address,0x01487ab11d90
Stoichiometry,gthrd_c + mthgxl_c --> lgt_S_c Reduced-glutathione + Methylglyoxal --> R-S-Lactoylglutathione
GPR,NGO08805
Lower bound,0.0
Upper bound,1000.0


In [224]:
#The cytochrome oxidation was flipped in the original CYOX reaction. Correcting here. 
#based on other examples from Bigg http://bigg.ucsd.edu/models/iAF987/reactions/CYOO2pp
#Additionally, no hydrogens were exported to the periplasm. This is also corrected here. 
r=model.reactions.get_by_id('CYOX')
model.metabolites.ficytC_p.charge=3
model.metabolites.focytC_p.charge=2
h_p=model.metabolites.h_p
ficytC_p =model.metabolites.ficytC_p
focytC_p=model.metabolites.focytC_p

r.add_metabolites({
    focytC_p:-8,
    ficytC_p: 8,
    h_c: -4.0,
    h_p: 4.0,
      })

print(r.check_mass_balance())
r


{}


Reaction identifier,CYOX
Name,R_CYOX
Memory address,0x014878ceebe0
Stoichiometry,"4.0 focytC_p + 8.0 h_c + o2_c <=> 4.0 ficytC_p + 2.0 h2o_c + 4.0 h_p 4.0 ferrocytochrome, cytc + 8.0 H + O2 <=> 4.0 ferricytochrome, cytc + 2.0 H2O + 4.0 H"
GPR,NGO1371 and NGO1373 and NGO1374 and ( NGO1080 or NGO0101 )
Lower bound,-1000.0
Upper bound,1000.0


In [225]:
r=model.reactions.get_by_id('CYOX')
model.reactions.CYOX.lower_bound=0
r

Reaction identifier,CYOX
Name,R_CYOX
Memory address,0x014878ceebe0
Stoichiometry,"4.0 focytC_p + 8.0 h_c + o2_c --> 4.0 ficytC_p + 2.0 h2o_c + 4.0 h_p 4.0 ferrocytochrome, cytc + 8.0 H + O2 --> 4.0 ferricytochrome, cytc + 2.0 H2O + 4.0 H"
GPR,NGO1371 and NGO1373 and NGO1374 and ( NGO1080 or NGO0101 )
Lower bound,0
Upper bound,1000.0


In [227]:
#Cytochrome oxidation was also flipped in the original UCYR reaction. Corrected here. 
#based on other examples from Bigg http://bigg.ucsd.edu/models/iAF987/reactions/CYTMQOR3pp
r=model.reactions.UCYR
r.add_metabolites({
    focytC_p:4,
    ficytC_p: -4,
      })

print(r.check_mass_balance())
r


{}


Reaction identifier,UCYR
Name,R_UCYR
Memory address,0x01487ac06880
Stoichiometry,"2.0 ficytC_p + q8h2_c <=> 2.0 focytC_p + 2.0 h_p + q8_c 2.0 ferricytochrome, cytc + Ubiquinol-8 <=> 2.0 ferrocytochrome, cytc + 2.0 H + Ubiquinone-8"
GPR,NGO2031 and NGO2030 and NGO2029 and ( NGO1080 or NGO0101 or Orphan )
Lower bound,-1000.0
Upper bound,1000.0


In [228]:
r=model.reactions.get_by_id('IGPDH')
model.metabolites.imacp_c.charge=-2
print(r.check_mass_balance())
r

{}


Reaction identifier,IGPDH
Name,R_IGPDH
Memory address,0x01487ab04490
Stoichiometry,eig3p_c --> h2o_c + imacp_c D-erythro-1-Imidazol-4-yl-glycerol-3-phosphate --> H2O + 3-Imidazol-4-yl-2-oxopropyl-phosphate
GPR,NGO1242
Lower bound,0.0
Upper bound,1000.0


In [229]:
#The original NO2R was cytochrome oxidations were flipped. Corrected here. 
r=model.reactions.get_by_id('NO2R')
r.name='nitric-oxide:ferricytochrome-c oxidoreductase'
ficytC_p =model.metabolites.ficytC_p
focytC_p=model.metabolites.focytC_p

r.add_metabolites({
    focytC_p:2,
    ficytC_p: -2,
      })

print(r.check_mass_balance())
r

{}


Reaction identifier,NO2R
Name,nitric-oxide:ferricytochrome-c oxidoreductase
Memory address,0x01487ab38970
Stoichiometry,"ficytC_p + h2o_p + no_p <=> focytC_p + 2.0 h_p + no2_p ferricytochrome, cytc + H2O + Nitric-oxide <=> ferrocytochrome, cytc + 2.0 H + Nitrite"
GPR,NGO1276 and NGO1328
Lower bound,-1000.0
Upper bound,1000.0


In [230]:
#The original NOR was cytochrome oxidations were flipped. Corrected here. 

r=model.reactions.get_by_id('NOR')
r.name='nitrous-oxide:ferricytochrome-c oxidoreductase'
ficytC_p =model.metabolites.ficytC_p
focytC_p=model.metabolites.focytC_p
r.add_metabolites({
    focytC_p:-4,
    ficytC_p: 4,
      })

print(r.check_mass_balance())
r

{}


Reaction identifier,NOR
Name,nitrous-oxide:ferricytochrome-c oxidoreductase
Memory address,0x01487ab381f0
Stoichiometry,"2.0 focytC_p + 2.0 h_c + 2.0 no_p <=> 2.0 ficytC_p + h2o_p + n2o_p 2.0 ferrocytochrome, cytc + 2.0 H + 2.0 Nitric-oxide <=> 2.0 ferricytochrome, cytc + H2O + Nitrous-oxide"
GPR,NGO1275 and NGO1328
Lower bound,-1000.0
Upper bound,1000.0


In [231]:
#Check Gc model ability to grow anaerobically on nitrite. 
def anaerobic_rpmi(model):
    for reaction in model.reactions:
        if 'EX_' in  reaction.id:
            reaction.lower_bound=0
        if reaction.id in rpmi_req:
            reaction.lower_bound = -10.
        if reaction.id == 'EX_o2_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_glc_D_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_lac_L_e_':
            reaction.lower_bound = -10.
        if reaction.id == 'EX_no2_e_':
            reaction.lower_bound = -10. 

anaerobic_rpmi(model)
print(model.slim_optimize())
print(model.metabolites.no_c.summary())

0.7097088373083901
no_c
====
Formula: NO

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []


In [232]:
r=model.reactions.CYPOX
print(r.check_mass_balance())
r
#Requires a cytochrome so as reactants, so can't be balanced here. 

{'charge': -2.0}


Reaction identifier,CYPOX
Name,Cytochrome C Peroxidase
Memory address,0x01487ac3adc0
Stoichiometry,h2o2_c + 2.0 h_c --> 2.0 h2o_c Hydrogen-peroxide + 2.0 H --> 2.0 H2O
GPR,NGO1769 and NGO0994
Lower bound,0.0
Upper bound,1000.0


In [233]:
r=model.reactions.HPYRRx
model.metabolites.glyc_R_c.charge=-1
print(r.check_mass_balance())
r

{}


Reaction identifier,HPYRRx
Name,R_HPYRRx
Memory address,0x01487aafd460
Stoichiometry,h_c + hpyr_c + nadh_c --> glyc_R_c + nad_c H + Hydroxypyruvate + Nicotinamide-adenine-dinucleotide--reduced --> R-Glycerate + Nicotinamide-adenine-dinucleotide
GPR,NGO2043
Lower bound,0.0
Upper bound,1000.0


In [234]:
r=model.reactions.Malt1pp
model.metabolites.mal_L_p.charge=-2
print(r.check_mass_balance())
r

{}


Reaction identifier,Malt1pp
Name,R_Malt1pp
Memory address,0x01487ab18dc0
Stoichiometry,mal_L_p + na1_p --> mal_L_c + na1_c L-Malate + Sodium --> L-Malate + Sodium
GPR,NGO1485 or NGO0377 or NGO1957
Lower bound,0.0
Upper bound,1000.0


In [235]:
r=model.reactions.MI1PP
model.metabolites.mi1p_D_c.charge=-2
print(r.check_mass_balance())
r

{}


Reaction identifier,MI1PP
Name,R_MI1PP
Memory address,0x01487ab24f70
Stoichiometry,h2o_c + mi1p_D_c --> inost_c + pi_c H2O + 1D-myo-Inositol-1-phosphate --> myo-Inositol + Phosphate
GPR,NGO2161 or NGO0671
Lower bound,0.0
Upper bound,1000.0


In [236]:
r=model.reactions.get_by_id('3HAD140')
#Update charge from bigg
model.metabolites.get_by_id('tmrs2eACP_c').charge=-1
print(r.check_mass_balance())
r


{}


Reaction identifier,3HAD140
Name,R_3HAD140
Memory address,0x014879582670
Stoichiometry,3hmrsACP_c --> h2o_c + tmrs2eACP_c R-3-Hydroxytetradecanoyl-acyl-carrier-protein --> H2O + trans-Tetradec-2-enoyl-acyl-carrier-protein
GPR,NGO1804
Lower bound,0.0
Upper bound,1000.0


In [237]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv('imbalancesremaining.csv')
mass_charge

63 63 63


,Reactions,Charge,Masses
0,3OAS161a,2.0,"{'C': -12.0, 'H': -21.0, 'N': -2.0, 'O': -9.0,..."
1,ADPT,-1.0,{'H': -1.0}
2,AGPAT161a,-2.0,{}
3,AGPAT181,-2.0,{}
4,BTS4,,{'S': 1.0}
...,...,...,...
58,PTHPS,,"{'C': -9.0, 'H': -11.0, 'N': -5.0, 'O': -3.0}"
59,CPH4S,1.0,"{'C': -7.0, 'H': -8.0, 'N': -5.0, 'O': -3.0}"
60,CDGS,,"{'H': 3.0, 'N': 1.0}"
61,CDGR,-1.0,{'H': -5.0}


In [238]:
#Added ACP and co2 to this reaction, as is seen in 3OAS161 for mass balance. 
r=model.reactions.get_by_id('3OAS161a')
ACP_c=model.metabolites.get_by_id('ACP_c')
co2_c=model.metabolites.get_by_id('co2_c')
r.add_metabolites({
    ACP_c:1,
    co2_c:1
      })
model.metabolites.get_by_id('3ocpalm5eACP_c').charge=-1
print(r.check_mass_balance())
r


{}


Reaction identifier,3OAS161a
Name,R_3OAS161a
Memory address,0x014878d20ee0
Stoichiometry,cmrs3eACP_c + h_c + malACP_c --> 3ocpalm5eACP_c + ACP_c + co2_c cis-Tetradec-3-enoyl-acyl-carrier-protein + H + Malonyl-acyl-carrier-protein --> 3-oxo-cis-palm-5-eoyl-acyl-carrier-protein + acyl-carrier-protein + CO2
GPR,NGO1763
Lower bound,0.0
Upper bound,1000.0


In [239]:
r=model.reactions.get_by_id('3OAR161x')
model.metabolites.get_by_id('3hcpalm5eACP_c').charge=-1
print(r.check_mass_balance())
r

{}


Reaction identifier,3OAR161x
Name,R_3OAR161x
Memory address,0x014879573f70
Stoichiometry,3ocpalm5eACP_c + h_c + nadph_c --> 3hcpalm5eACP_c + nadp_c 3-oxo-cis-palm-5-eoyl-acyl-carrier-protein + H + Nicotinamide-adenine-dinucleotide-phosphate--reduced --> M_3hcpalm5eACP_c + Nicotinamide-adenine-dinucleotide-phosphate
GPR,NGO2163 or NGO2163
Lower bound,0.0
Upper bound,1000.0


In [240]:
r=model.reactions.get_by_id('3HAD161x')
model.metabolites.get_by_id('t3c5palmeACP_c').charge=-1
print(r.check_mass_balance())
r

{}


Reaction identifier,3HAD161x
Name,R_3HAD161x
Memory address,0x014879597550
Stoichiometry,"3hcpalm5eACP_c --> h2o_c + t3c5palmeACP_c M_3hcpalm5eACP_c --> H2O + Abbreviated name t3c5palmeACP_c trans-3-cis-5-palmitoleoyl-acyl-carrier-protein t3c5palmeACP_c trans-3-cis-5-palmitoleoyl-acyl-carrier-protein Name: Full name, dtype: object"
GPR,NGO1804
Lower bound,0.0
Upper bound,1000.0


In [241]:
r=model.reactions.get_by_id('AGPAT161a')
model.metabolites.get_by_id('1hdect9eg3p_c').charge=-2
print(r.check_mass_balance())
r

{}


Reaction identifier,AGPAT161a
Name,R_AGPAT161a
Memory address,0x014878d37ca0
Stoichiometry,1hdect9eg3p_c + hdet9ACP_c --> ACP_c + pa161_c 1-hexadec-trans-9-enoyl-sn-glycerol-3-phosphate + trans-hexadec-9-enoyl-acyl-carrier-protein-n-C161 --> acyl-carrier-protein + 1-2-dihexadec-trans-9-enoyl-sn-glycerol-3-phosphate
GPR,NGO2069 or NGO0611
Lower bound,0.0
Upper bound,1000.0


In [242]:
r=model.reactions.get_by_id('AGPAT181')
model.metabolites.get_by_id('1odec7eg3p_c').charge= -2
print(r.check_mass_balance())
r

{}


Reaction identifier,AGPAT181
Name,R_AGPAT181
Memory address,0x014878cf5760
Stoichiometry,1odec7eg3p_c + oct7eACP_c --> ACP_c + pa181_c 1-octadec-7-enoyl-sn-glycerol-3-phosphate + cis-octadec-7-enoyl-acyl-carrier-protein-n-C181 --> acyl-carrier-protein + 1-2-dioctadec-7-enoyl-sn-glycerol-3-phosphate
GPR,NGO2069 or NGO0611
Lower bound,0.0
Upper bound,1000.0


In [243]:
#From Bigg for DASYN161
r=model.reactions.get_by_id('DASYN161a')
model.metabolites.get_by_id('cdp161_c').charge=-2
model.metabolites.get_by_id('cdp161_c').formula= 'C44H75N3O15P2'
print(r.check_mass_balance())
r

{}


Reaction identifier,DASYN161a
Name,R_DASYN161a
Memory address,0x014878cbfac0
Stoichiometry,ctp_c + h_c + pa161_c --> cdp161_c + ppi_c CTP + H + 1-2-dihexadec-trans-9-enoyl-sn-glycerol-3-phosphate --> cdp-1-hexadec-trans-9-enoyl-2-octadeca-cis-7-enoyl-sn-glycerol + Diphosphate
GPR,NGO1798
Lower bound,0.0
Upper bound,1000.0


In [244]:
r=model.reactions.get_by_id('DASYN181')
model.metabolites.get_by_id('cdpdodec7eg_c').charge=-2
print(r.check_mass_balance())
r

{}


Reaction identifier,DASYN181
Name,R_DASYN181
Memory address,0x014879ad6df0
Stoichiometry,ctp_c + h_c + pa181_c --> cdpdodec7eg_c + ppi_c CTP + H + 1-2-dioctadec-7-enoyl-sn-glycerol-3-phosphate --> CDP-1-2-dioctadec-7-enoylglycerol + Diphosphate
GPR,NGO1798
Lower bound,0.0
Upper bound,1000.0


In [245]:
r=model.reactions.get_by_id('EAR161a')
#Based on Bigg charge for similar molecules in EAR161
model.metabolites.get_by_id('h5deACP_c').charge=-1
print(r.check_mass_balance())
r

{}


Reaction identifier,EAR161a
Name,R_EAR161a
Memory address,0x014879ab0a60
Stoichiometry,h_c + nadh_c + t3c5palmeACP_c --> h5deACP_c + nad_c H + Nicotinamide-adenine-dinucleotide--reduced + Abbreviated name t3c5palmeACP_c trans-3-cis-5-palmitoleoyl-acyl-carrier-protein t3c5palmeACP_c trans-3-cis-5-palmitoleoyl-acyl-carrier-protein Name:...
GPR,NGO1666
Lower bound,0.0
Upper bound,1000.0


In [246]:
r=model.reactions.get_by_id('3OAS181a')
#Based on Bigg charge for similar molecules in 3OAS181
model.metabolites.get_by_id('3ocvac7eACP_c').charge=-1
print(r.check_mass_balance())
r


{}


Reaction identifier,3OAS181a
Name,R_3OAS181a
Memory address,0x014879573610
Stoichiometry,h5deACP_c + h_c + malACP_c --> 3ocvac7eACP_c + ACP_c + co2_c cis-hexadec-5-enoyl-acyl-carrier-protein-n-C161 + H + Malonyl-acyl-carrier-protein --> 3-oxo-cis-vacc-7-enoyl-acyl-carrier-protein + acyl-carrier-protein + CO2
GPR,NGO1763
Lower bound,0.0
Upper bound,1000.0


In [247]:
r=model.reactions.get_by_id('3OAR181x')
#Based on Bigg charge for similar molecules in 3OAR181x
model.metabolites.get_by_id('3hcvac7eACP_c').charge=-1
print(r.check_mass_balance())
r

{}


Reaction identifier,3OAR181x
Name,R_3OAR181x
Memory address,0x014879577f10
Stoichiometry,3ocvac7eACP_c + h_c + nadph_c --> 3hcvac7eACP_c + nadp_c 3-oxo-cis-vacc-7-enoyl-acyl-carrier-protein + H + Nicotinamide-adenine-dinucleotide-phosphate--reduced --> R-3-hydroxy-cis-vacc-7-enoyl-acyl-carrier-protein + Nicotinamide-adenine-dinucleotide-...
GPR,NGO2163 or NGO2163
Lower bound,0.0
Upper bound,1000.0


In [248]:
r=model.reactions.get_by_id('3HAD181a')
#Based on Bigg charge for similar molecules in 3HAD181
model.metabolites.get_by_id('t3c7vaceACP_c').charge=-1
print(r.check_mass_balance())
r


{}


Reaction identifier,3HAD181a
Name,R_3HAD181a
Memory address,0x014879597ee0
Stoichiometry,3hcvac7eACP_c --> h2o_c + t3c7vaceACP_c R-3-hydroxy-cis-vacc-7-enoyl-acyl-carrier-protein --> H2O + trans-3-cis-7-vacceoyl-acyl-carrier-protein
GPR,NGO1804
Lower bound,0.0
Upper bound,1000.0


In [249]:
r=model.reactions.get_by_id('MALtex')
model.metabolites.get_by_id('mal_L_e').charge=-2
print(r.check_mass_balance())
r

{}


Reaction identifier,MALtex
Name,R_MALtex
Memory address,0x01487ab18f10
Stoichiometry,mal_L_e <=> mal_L_p L-Malate <=> L-Malate
GPR,NGO1812
Lower bound,-1000.0
Upper bound,1000.0


In [250]:
r=model.reactions.get_by_id('pa161abcpp')
model.metabolites.get_by_id('pa161_p').charge=-2
print(r.check_mass_balance())
r



{}


Reaction identifier,pa161abcpp
Name,R_pa161abcpp
Memory address,0x01487ab4bc10
Stoichiometry,atp_c + h2o_c + pa161_c --> adp_c + h_c + pa161_p + pi_c ATP + H2O + 1-2-dihexadec-trans-9-enoyl-sn-glycerol-3-phosphate --> ADP + H + 1-2-dihexadec-trans-9-enoyl-sn-glycerol-3-phosphate + Phosphate
GPR,NGO1682 and NGO1801
Lower bound,0.0
Upper bound,1000.0


In [251]:
r=model.reactions.get_by_id('pa181abcpp')
model.metabolites.get_by_id('pa181_p').charge=-2
print(r.check_mass_balance())
r

{}


Reaction identifier,pa181abcpp
Name,R_pa181abcpp
Memory address,0x01487ab4b6a0
Stoichiometry,atp_c + h2o_c + pa181_c --> adp_c + h_c + pa181_p + pi_c ATP + H2O + 1-2-dioctadec-7-enoyl-sn-glycerol-3-phosphate --> ADP + H + 1-2-dioctadec-7-enoyl-sn-glycerol-3-phosphate + Phosphate
GPR,NGO1682 and NGO1801
Lower bound,0.0
Upper bound,1000.0


In [252]:
r=model.reactions.get_by_id('pgp161abcpp')
model.metabolites.get_by_id('pgp161_p').charge=-3
model.metabolites.get_by_id('pgp161_c').charge=-3
print(r.check_mass_balance())
r



{}


Reaction identifier,pgp161abcpp
Name,R_pgp161abcpp
Memory address,0x01487ab56d60
Stoichiometry,atp_c + h2o_c + pgp161_c --> adp_c + h_c + pgp161_p + pi_c ATP + H2O + Phosphatidylglycerophosphate-1-hexadec-trans-9-enoyl-2-octadeca-cis-7-enoyl-n-161a --> ADP + H + Phosphatidylglycerophosphate-1-hexadec-trans-9-enoyl-2-octadeca-cis-7-enoyl-n-161a +...
GPR,NGO1682 and NGO1801
Lower bound,0.0
Upper bound,1000.0


In [253]:
r=model.reactions.get_by_id('pgp181abcpp')
model.metabolites.get_by_id('pgp181_p').charge=-3
model.metabolites.get_by_id('pgp181_c').charge=-3
print(r.check_mass_balance())
r

{}


Reaction identifier,pgp181abcpp
Name,R_pgp181abcpp
Memory address,0x01487ab59820
Stoichiometry,atp_c + h2o_c + pgp181_c --> adp_c + h_c + pgp181_p + pi_c ATP + H2O + Phosphatidylglycerophosphate-dioctadec-7-enoyl-n-C181 --> ADP + H + Phosphatidylglycerophosphate-dioctadec-7-enoyl-n-C181 + Phosphate
GPR,NGO1682 and NGO1801
Lower bound,0.0
Upper bound,1000.0


In [254]:
r=model.reactions.get_by_id('PGSA161a')
r.add_metabolites({
    h_c:1,
      })
print(r.check_mass_balance())
r


{}


Reaction identifier,PGSA161a
Name,R_PGSA161a
Memory address,0x01487ab62b50
Stoichiometry,cdp161_c + glyc3p_c --> cmp_c + h_c + pgp161_c cdp-1-hexadec-trans-9-enoyl-2-octadeca-cis-7-enoyl-sn-glycerol + Glycerol-3-phosphate --> CMP + H + Phosphatidylglycerophosphate-1-hexadec-trans-9-enoyl-2-octadeca-cis-7-enoyl-n-161a
GPR,NGO1247
Lower bound,0.0
Upper bound,1000.0


In [255]:
r=model.reactions.get_by_id('PLIPA1A161a')
model.metabolites.get_by_id('hdcat9_p').charge=-1
model.metabolites.get_by_id('2hdect9g3p_p').charge=-1
print(r.check_mass_balance())
r



{}


Reaction identifier,PLIPA1A161a
Name,R_PLIPA1A161a
Memory address,0x01487ab66a00
Stoichiometry,h2o_p + pa161_p --> 2hdect9g3p_p + hdcat9_p H2O + 1-2-dihexadec-trans-9-enoyl-sn-glycerol-3-phosphate --> 2-hexadec-trans-9-enoyl-sn-glycerol-3-phosphate + Hexadeca-trans-9-enoyloate-n-C161
GPR,NGO1492
Lower bound,0.0
Upper bound,1000.0


In [256]:
r=model.reactions.get_by_id('PLIPA1A181pp')
model.metabolites.get_by_id('2odec7eg3p_p').charge=-1
print(r.check_mass_balance())
r


{}


Reaction identifier,PLIPA1A181pp
Name,R_PLIPA1A181pp
Memory address,0x01487ab66e50
Stoichiometry,h2o_p + pa181_p --> 2odec7eg3p_p + ocdcea_p H2O + 1-2-dioctadec-7-enoyl-sn-glycerol-3-phosphate --> 2-octadec-7-enoyl-sn-glycerol-3-phosphate + octadecenoate-n-C181
GPR,NGO1492
Lower bound,0.0
Upper bound,1000.0


In [257]:
r=model.reactions.get_by_id('2AGpa161tipp')
model.metabolites.get_by_id('2hdect9g3p_c').charge=-1
print(r.check_mass_balance())
r
#Duplicate reaction with 2AGPA161tipp. Removing
model.remove_reactions('2AGpa161tipp')

{}


C:\Users\Aimee\anaconda3\lib\site-packages\cobra\core\model.py:727: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [258]:
r=model.reactions.get_by_id('2AGPA161tipp')
model.metabolites.get_by_id('2hdect9g3p_c').charge=-1
print(r.check_mass_balance())
r

{}


Reaction identifier,2AGPA161tipp
Name,R_2AGPA161tipp
Memory address,0x014879582070
Stoichiometry,2hdect9g3p_p --> 2hdect9g3p_c 2-hexadec-trans-9-enoyl-sn-glycerol-3-phosphate --> 2-hexadec-trans-9-enoyl-sn-glycerol-3-phosphate
GPR,NGO1682 and NGO1801
Lower bound,0.0
Upper bound,1000.0


In [259]:
r=model.reactions.get_by_id('2AGPA181tipp')
model.metabolites.get_by_id('2odec7eg3p_c').charge=-1
print(r.check_mass_balance())
r

{}


Reaction identifier,2AGPA181tipp
Name,R_2AGPA181tipp
Memory address,0x014879596fa0
Stoichiometry,2odec7eg3p_p --> 2odec7eg3p_c 2-octadec-7-enoyl-sn-glycerol-3-phosphate --> 2-octadec-7-enoyl-sn-glycerol-3-phosphate
GPR,NGO1682 and NGO1801
Lower bound,0.0
Upper bound,1000.0


In [260]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv('imbalancesremaining.csv')
mass_charge

44 44 44


,Reactions,Charge,Masses
0,ADPT,-1.0,{'H': -1.0}
1,BTS4,,{'S': 1.0}
2,CYTCS,5.0,"{'C': 8.0, 'H': 14.0, 'N': 4.0, 'O': 4.0, 'S':..."
3,DHAPtex,-2.0,{}
4,DHORD2,-1.0,{}
5,DHORTS,-1.0,{}
6,DDPGA,-2.0,{}
7,HGFE,-8.0,{}
8,EHGLAT,1.0,{}
9,MAc2,6.0,{}


In [261]:
r=model.reactions.get_by_id('PLIPA1E161a')
model.metabolites.get_by_id('pe161_p').formula='C37H70N1O8P1'
r.add_metabolites({
    h_c:1,
      })
print(r.check_mass_balance())
r

{}


Reaction identifier,PLIPA1E161a
Name,R_PLIPA1E161a
Memory address,0x01487ab6afd0
Stoichiometry,h2o_p + pe161_p --> 2agpe161_p + h_c + hdcat9_p H2O + M_pe161_p --> 2-Acyl-sn-glycero-3-phosphoethanolamine-n-C161(t9) + H + Hexadeca-trans-9-enoyloate-n-C161
GPR,NGO1492
Lower bound,0.0
Upper bound,1000.0


In [262]:
r=model.reactions.get_by_id('pe161abcpp')
model.metabolites.get_by_id('pe161_c').formula='C37H70N1O8P1'

print(r.check_mass_balance())
r


{}


Reaction identifier,pe161abcpp
Name,R_pe161abcpp
Memory address,0x01487ab52fd0
Stoichiometry,atp_c + h2o_c + pe161_c --> adp_c + h_c + pe161_p + pi_c ATP + H2O + M_pe161_c --> ADP + H + M_pe161_p + Phosphate
GPR,NGO1682 and NGO1801
Lower bound,0.0
Upper bound,1000.0


In [263]:
r=model.reactions.get_by_id('PSD161a')
model.metabolites.get_by_id('ps161_c').formula='C38H69N1O10P1'

print(r.check_mass_balance())
r



{}


Reaction identifier,PSD161a
Name,R_PSD161a
Memory address,0x01487ab8a8b0
Stoichiometry,h_c + ps161_c --> co2_c + pe161_c H + M_ps161_c --> CO2 + M_pe161_c
GPR,NGO1206
Lower bound,0.0
Upper bound,1000.0


In [264]:
r=model.reactions.get_by_id('MAc2')
model.metabolites.murein4px4px4pOAc_p.charge=-6
print(r.check_mass_balance())
r

{}


Reaction identifier,MAc2
Name,R_MAc2
Memory address,0x01487ab14be0
Stoichiometry,accoa_c + h2o_p + murein4px4px4p_p --> coa_c + murein4px4px4pOAc_p Acetyl-CoA + H2O + three-disacharide-linked-murein-units-tetrapeptide-crosslinked-tetrapeptide-A2pm-D-ala--tetrapeptide-corsslinked-tetrapeptide-A2pm-D-ala--middle-of-chain --> Coenzyme-A + three-...
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,1000.0


In [265]:
r=model.reactions.get_by_id('MAc4')
model.metabolites.murein4p4pOAc_p.charge=-4
print(r.check_mass_balance())
r

{}


Reaction identifier,MAc4
Name,R_MAc4
Memory address,0x01487ab14fa0
Stoichiometry,accoa_c + h2o_p + murein4p4p_p --> coa_c + murein4p4pOAc_p Acetyl-CoA + H2O + two-linked-disacharide-tetrapeptide-murein-units-uncrosslinked-middle-of-chain --> Coenzyme-A + two-linked-disacharide-tetrapeptide-murein-units-uncrosslinked-middle-of-chain-O-...
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,1000.0


In [266]:
r=model.reactions.get_by_id('MAc5')
model.metabolites.murein4p4pdOAc_p.charge=-4
print(r.check_mass_balance())
r

{}


Reaction identifier,MAc5
Name,R_MAc5
Memory address,0x01487ab14970
Stoichiometry,accoa_c + h2o_p + murein4p4pOAc_p --> coa_c + murein4p4pdOAc_p Acetyl-CoA + H2O + two-linked-disacharide-tetrapeptide-murein-units-uncrosslinked-middle-of-chain-O-acytelated --> Coenzyme-A + two-linked-disacharide-tetrapeptide-murein-units-uncrosslinked-...
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,1000.0


In [267]:
r=model.reactions.get_by_id('MAc6')
model.metabolites.murein4p3pOAc_p.charge=-4
print(r.check_mass_balance())
r

{}


Reaction identifier,MAc6
Name,R_MAc6
Memory address,0x01487ab14d00
Stoichiometry,accoa_c + h2o_p + murein4p3p_p --> coa_c + murein4p3pOAc_p Acetyl-CoA + H2O + two-linked-disacharide-tetrapeptide-and-tripeptide-murein-units-uncrosslinked-middle-of-chain --> Coenzyme-A + two-linked-disacharide-tetrapeptide-and-tripeptide-murein-units-...
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,1000.0


In [268]:
r=model.reactions.get_by_id('MAc7')
model.metabolites.murein4px4px4pdOAc_p.charge=-6
print(r.check_mass_balance())
r

{}


Reaction identifier,MAc7
Name,R_MAc7
Memory address,0x01487ab11e50
Stoichiometry,accoa_c + h2o_p + murein4px4px4pOAc_p --> coa_c + murein4px4px4pdOAc_p Acetyl-CoA + H2O + three-disacharide-linked-murein-units-tetrapeptide-crosslinked-tetrapeptide-A2pm-D-ala--tetrapeptide-corsslinked-tetrapeptide-A2pm-D-ala--middle-of-chain-O_acytelated -->...
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,1000.0


In [269]:
r=model.reactions.get_by_id('MAc8')
model.metabolites.murein4px4px4ptOAc_p.charge=-6
print(r.check_mass_balance())
r

{}


Reaction identifier,MAc8
Name,R_MAc8
Memory address,0x01487ab14e80
Stoichiometry,accoa_c + h2o_p + murein4px4px4pdOAc_p --> coa_c + murein4px4px4ptOAc_p Acetyl-CoA + H2O + M_murein4px4px4pdOAc_p --> Coenzyme-A + M_murein4px4px4ptOAc_p
GPR,NGO0533 and NGO0534
Lower bound,0.0
Upper bound,1000.0


In [293]:
model.metabolites.hdceap_c.charge=-1
model.metabolites.ocdceap_c.charge=-1

In [270]:
#Remaining imbalances are ok and due to proteinaceous components or reaction simplifications. 

In [294]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in model.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv('imbalancesremaining.csv')
mass_charge

35 35 35


,Reactions,Charge,Masses
0,ADPT,-1.0,{'H': -1.0}
1,BTS4,,{'S': 1.0}
2,CYTCS,5.0,"{'C': 8.0, 'H': 14.0, 'N': 4.0, 'O': 4.0, 'S':..."
3,DHAPtex,-2.0,{}
4,DHORD2,-1.0,{}
5,DHORTS,-1.0,{}
6,DDPGA,-2.0,{}
7,HGFE,-8.0,{}
8,EHGLAT,1.0,{}
9,METS,-1.0,{'H': -1.0}


In [303]:
r=model.reactions.ADPT
r.check_mass_balance()

{'charge': -1.0, 'H': -1.0}

In [302]:
r=test.reactions.ADPT
r.check_mass_balance()

{}

In [301]:
print(model.metabolites.ppi_c.charge)
print(test.metabolites.ppi_c.charge)
print(model.metabolites.ppi_c.formula)
print(test.metabolites.ppi_c.formula)

-3
-3
HO7P2
HO7P2


In [273]:
test=read_model('C:/Users/Aimee/Documents/Metabolic_Modeling/organized/models/annotatedGCmodel.json')

In [274]:
test

Name,NGO_558
Memory address,0x01487c59bee0
Number of metabolites,1263
Number of reactions,1423
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"periplasm, cytoplasm, extracellular"


In [275]:
#After bulk removing cofactors, what reaction imbalances remain?
reactions = []
charge = []
mass = []
for react in test.reactions:
    result = react.check_mass_balance()
    if 'EX_' in  react.id:
        continue
    if 'Nm_' in  react.id:
        continue
    if 'DM_' in  react.id:
        continue 
    if react.id in biomass_reactions:
        continue
    if len(result) == 0:
        continue
    else: 
        reactions.append(react.id)
        masses = {}
        for key in result.keys():
            if key == 'charge':
                charge.append(result['charge'])
            else:
                masses[key] = result[key]
        if 'charge' not in result.keys():
            charge.append('')
        mass.append(masses)
print(len(reactions), len(charge), len(mass))

mass_charge_dict = {'Reactions': reactions, 'Charge': charge, 'Masses': mass}
mass_charge = pandas.DataFrame(mass_charge_dict)
mass_charge.to_csv('imbalancesremaining.csv')
mass_charge

8 8 8


,Reactions,Charge,Masses
0,BTS4,,{'S': 1.0}
1,CYTCS,5.0,"{'C': 8.0, 'H': 14.0, 'N': 4.0, 'O': 4.0, 'S':..."
2,HGFE,-8.0,{}
3,MDDCP6pp,6.0,"{'H': 3.0, 'N': -6.0}"
4,biomass,1.508643,"{'C': -41.660179302281534, 'H': 26.21141557175..."
5,PHEMEFErel,7.0,{}
6,PMCOAS,-1.0,"{'C': 7.0, 'H': 9.0, 'O': 3.0}"
7,CYPOX,-2.0,{}


In [137]:
print("Ngo_533- metabolites :", len(model.metabolites))
print("Ngo_533- reactions :", len(model.reactions))
print("Ngo_533- genes :", len(model.genes))

Ngo_533- metabolites : 1341
Ngo_533- reactions : 1432
Ngo_533- genes : 559


In [285]:
model.objective='Nm_Ess_biomass'
rpmi(model)
print ('RPMI',model.slim_optimize())
mdm(model)
print ('MDM', model.slim_optimize())
rpmi_molarity(model)
print ('RPMI_molarity', model.slim_optimize())
complete(model)
print('Complete media', model.slim_optimize())

RPMI 1.1805315902906208
MDM 1.241330539428735
RPMI_molarity 0.27174347285720757
Complete media 1.8729354491783996


In [286]:
r=model.reactions.get_by_id('Nm_Ess_biomass')
pa120_p=model.metabolites.get_by_id('pa120_p')
pa140_p=model.metabolites.get_by_id('pa140_p')
pa160_p=model.metabolites.get_by_id('pa160_p')
pa180_p=model.metabolites.get_by_id('pa180_p')
pa141_p=model.metabolites.get_by_id('pa141_p')
pa161_p=model.metabolites.get_by_id('pa161_p')
pa181_p=model.metabolites.get_by_id('pa181_p')
pe120_p=model.metabolites.get_by_id('pe120_p')
pe140_p=model.metabolites.get_by_id('pe140_p')
pe160_p=model.metabolites.get_by_id('pe160_p')
pe180_p=model.metabolites.get_by_id('pe180_p')
pe141_p=model.metabolites.get_by_id('pe141_p')
pe161_p=model.metabolites.get_by_id('pe161_p')
pe181_p=model.metabolites.get_by_id('pe181_p')
pg120_p=model.metabolites.get_by_id('pg120_p')
pg140_p=model.metabolites.get_by_id('pg140_p')
pg160_p=model.metabolites.get_by_id('pg160_p')
pg180_p=model.metabolites.get_by_id('pg180_p')
pg141_p=model.metabolites.get_by_id('pg141_p')
pg161a_p=model.metabolites.get_by_id('pg161_p')
pg181c_p=model.metabolites.get_by_id('pg181_p')
los_GC_c=model.metabolites.get_by_id('los_GC_c')
murein4px4px4pOAc_p=model.metabolites.get_by_id('murein4px4px4pOAc_p')
murein4p4p4pOAc_p=model.metabolites.get_by_id('murein4p4p4pOAc_p')
asp_L_c=model.metabolites.get_by_id('asp_L_c')
glu_L_c=model.metabolites.get_by_id('glu_L_c')
ser_L_c=model.metabolites.get_by_id('ser_L_c')
his_L_c=model.metabolites.get_by_id('his_L_c')
gly_c=model.metabolites.get_by_id('gly_c')
thr_L_c=model.metabolites.get_by_id('thr_L_c')
ala_L_c=model.metabolites.get_by_id('ala_L_c')
arg_L_c=model.metabolites.get_by_id('arg_L_c')
tyr_L_c=model.metabolites.get_by_id('tyr_L_c')
val_L_c=model.metabolites.get_by_id('val_L_c')
met_L_c=model.metabolites.get_by_id('met_L_c')
phe_L_c=model.metabolites.get_by_id('phe_L_c')
ile_L_c=model.metabolites.get_by_id('ile_L_c')
leu_L_c=model.metabolites.get_by_id('leu_L_c')
lys_L_c=model.metabolites.get_by_id('lys_L_c')
pro_L_c=model.metabolites.get_by_id('pro_L_c')
asn_L_c=model.metabolites.get_by_id('asn_L_c')
trp_L_c=model.metabolites.get_by_id('trp_L_c')
cys_L_c=model.metabolites.get_by_id('cys_L_c')
gln_L_c=model.metabolites.get_by_id('gln_L_c')
datp_c=model.metabolites.get_by_id('datp_c')
dttp_c=model.metabolites.get_by_id('dttp_c')
dgtp_c=model.metabolites.get_by_id('dgtp_c')
dctp_c=model.metabolites.get_by_id('dctp_c')
atp_c=model.metabolites.get_by_id('atp_c')
utp_c=model.metabolites.get_by_id('utp_c')
gtp_c=model.metabolites.get_by_id('gtp_c')
ctp_c=model.metabolites.get_by_id('ctp_c')
fthf_c=model.metabolites.get_by_id('10fthf_c')
ohph_c=model.metabolites.get_by_id('2ohph_c')
amet_c=model.metabolites.get_by_id('amet_c')
ca2_c=model.metabolites.get_by_id('ca2_c')
cl_c=model.metabolites.get_by_id('cl_c')
coa_c=model.metabolites.get_by_id('coa_c')
cobalt2_c=model.metabolites.get_by_id('cobalt2_c')
cu2_c=model.metabolites.get_by_id('cu2_c')
fad_c=model.metabolites.get_by_id('fad_c')
fe2_c=model.metabolites.get_by_id('fe2_c')
fe3_c=model.metabolites.get_by_id('fe3_c')
k_c=model.metabolites.get_by_id('k_c')
mg2_c=model.metabolites.get_by_id('mg2_c')
mlthf_c=model.metabolites.get_by_id('mlthf_c')
mn2_c=model.metabolites.get_by_id('mn2_c')
mobd_c=model.metabolites.get_by_id('mobd_c')
nad_c=model.metabolites.get_by_id('nad_c')
nadp_c=model.metabolites.get_by_id('nadp_c')
nh4_c=model.metabolites.get_by_id('nh4_c')
pheme_c=model.metabolites.get_by_id('pheme_c')
pydx5p_c=model.metabolites.get_by_id('pydx5p_c')
ribflv_c=model.metabolites.get_by_id('ribflv_c')
sheme_c=model.metabolites.get_by_id('sheme_c')
so4_c=model.metabolites.get_by_id('so4_c')
thf_c=model.metabolites.get_by_id('thf_c')
thmpp_c=model.metabolites.get_by_id('thmpp_c')
udcpdp_c=model.metabolites.get_by_id('udcpdp_c')
zn2_c=model.metabolites.get_by_id('zn2_c')
h_c=model.metabolites.get_by_id('h_c')
pi_c=model.metabolites.get_by_id('pi_c')
adp_c=model.metabolites.get_by_id('adp_c')
biomass=model.metabolites.get_by_id('biomass')
capsule_NM_e=model.metabolites.get_by_id('capsule_NM_e')
dna_c=model.metabolites.get_by_id('dna_c')
rna_c=model.metabolites.get_by_id('rna_c')
pepglycan_p=model.metabolites.get_by_id('pepglycan_p')
protein=model.metabolites.get_by_id('protein')
lipid_NM_p=model.metabolites.get_by_id('lipid_NM_p')
ptrc_c=model.metabolites.get_by_id('ptrc_c')


In [287]:
#remove Nm_biomass components
r=model.reactions.Nm_biomass
r.add_metabolites({
    fthf_c:0.000223,
    ohph_c:0.000223,
    amet_c:0.000223,
    atp_c:52.84,
    ca2_c:0.004737,
    capsule_NM_e:0.00146,
    cl_c:0.004737,
    coa_c:0.000576,
    cobalt2_c:0.003158,
    cu2_c:0.003158,
    dna_c:0.0798,
    fad_c:0.000223,
    fe2_c:0.007106,
    fe3_c:0.007106,
    h2o_c:52.84,
    k_c:0.1776,
    lipid_NM_p:0.1483,
    mg2_c:0.007895,
    mlthf_c:0.000223,
    mn2_c:0.003158,
    mobd_c:0.003158,
    nad_c:0.001831,
    nadp_c:0.000447,
    nh4_c:0.011843,
    pepglycan_p:0.1389,
    pheme_c:0.000223,
    protein:5.55,
    ptrc_c:0.0077376,
    pydx5p_c:0.000223,
    ribflv_c:0.000223,
    rna_c:0.4432,
    sheme_c:0.000223,
    so4_c:0.003948,
    thf_c:0.000223,
    thmpp_c:0.000223,
    udcpdp_c:0.000055,
    zn2_c:0.003158,
    h_c:-52.84,
    pi_c:-52.84,
    adp_c:-52.84,
    biomass:-1,})


In [288]:
#New biomass equation for Gc
r=model.reactions.Nm_biomass
r.add_metabolites({
    pa120_p:-0.000503555042155509,
    pa140_p:-0,
    pa160_p:-0.00207597350252432,
    pa180_p:-0.000290080640591335,
    pa141_p:-0.000371731800974898,
    pa161_p:-0.00564538392418283,
    pa181_p:-0.00216477129604764,
    pe120_p:-0.00234345993674653,
    pe140_p:-0.0025598346977071,
    pe160_p:-0.055748823259178,
    pe180_p:-0.00137834776769998,
    pe141_p:-0.00821977537556325,
    pe161_p:-0.0255682154771119,
    pe181_p:-0.0102822273478348,
    pg120_p:-0.000703620745734723,
    pg140_p:-3.03207570481577E-05,
    pg160_p:-0.0122303684865845,
    pg180_p:-0.000418944804169265,
    pg141_p:-0.00290595573235281,
    pg161a_p:-0.00862596402374158,
    pg181c_p:-0.00312454317572049,
    los_GC_c:-0.0129924495638587,
    murein4px4px4pOAc_p:-0.005556,
    murein4p4p4pOAc_p:-0.008334,
    asp_L_c:-0.328506488951245,
    glu_L_c:-0.417926718513411,
    ser_L_c:-0.298519204391151,
    his_L_c:-0.101130783859868,
    gly_c:-0.63200279142024,
    thr_L_c:-0.289017233514768,
    ala_L_c:-0.801362443841416,
    arg_L_c:-0.326223794247415,
    tyr_L_c:-0.142730093324994,
    val_L_c:-0.43926986171134,
    met_L_c:-0.102311261238582,
    phe_L_c:-0.199024522901412,
    ile_L_c:-0.291900141026265,
    leu_L_c:-0.472627376778854,
    lys_L_c:-0.351058182823571,
    pro_L_c:-0.246449661470245,
    asn_L_c:-0.251609961417047,
    trp_L_c:-0.0689480779843109,
    cys_L_c:-0.0606915932106025,
    gln_L_c:-0.246224295606094,
    datp_c:-0.0193660372420178,
    dttp_c:-0.0193660372420178,
    dgtp_c:-0.0205433507259598,
    dctp_c:-0.0205433507259598,
    atp_c:-0.108889121528625,
    utp_c:-0.108889121528625,
    gtp_c:-0.112753363956914,
    ctp_c:-0.112753363956914,
    fthf_c:-0.000223,
    ohph_c:-0.000223,
    amet_c:-0.000223,
    ca2_c:-0.004737,
    cl_c:-0.004737,
    coa_c:-0.000576,
    cobalt2_c:-0.003158,
    cu2_c:-0.003158,
    fad_c:-0.000223,
    fe2_c:-0.007106,
    fe3_c:-0.007106,
    k_c:-0.1776,
    mg2_c:-0.007895,
    mlthf_c:-0.000223,
    mn2_c:-0.003158,
    mobd_c:-0.003158,
    nad_c:-0.001831,
    nadp_c:-0.000447,
    nh4_c:-0.11843,
    pheme_c:-0.000223,
    pydx5p_c:-0.000223,
    ribflv_c:-0.000223,
    sheme_c:-0.000223,
    so4_c:-0.003948,
    thf_c:-0.000223,
    thmpp_c:-0.000223,
    udcpdp_c:-0.000055,
    zn2_c:-0.003158,
    atp_c: -52.84,
    h_c:52.84,
    pi_c:52.84,
    adp_c:52.84,
    biomass:1,
    })

r.name='biomass'
r.id='biomass'

In [289]:
model.objective='biomass'

In [290]:
rpmi(model)
print ('RPMI',model.slim_optimize())
mdm(model)
print ('MDM', model.slim_optimize())
rpmi_molarity(model)
print ('RPMI_molarity', model.slim_optimize())
complete(model)
print('Complete media', model.slim_optimize())

RPMI 1.5748164611774584
MDM 1.6558002724411476
RPMI_molarity 0.2849964780785233
Complete media 2.4998508320954747


In [291]:
model.remove_reactions('PLIPIDS')
model.remove_reactions('PLIPIDS_ESS')
model.remove_reactions('PEPGLY')
model.remove_reactions('PROTS')
model.remove_reactions('DNAS')
model.remove_reactions('RNAS')
model.remove_reactions('Nm_Ess_biomass')
model.remove_reactions('Nm_Baart_biomass')
model.remove_reactions('DM_CAPBIOMASS')
model.remove_reactions('DM_ESSBIOMASS')

print('Complete media', model.slim_optimize())

Complete media 2.499850832095472


C:\Users\Aimee\anaconda3\lib\site-packages\cobra\core\model.py:727: UserWarning: need to pass in a list
  warn("need to pass in a list")


In [292]:
model.metabolites.trp_L_c.annotation['bigg.metabolite'] = 'trp__L'

In [293]:
rpmi(model)
print ('RPMI',model.slim_optimize())
mdm(model)
print ('MDM', model.slim_optimize())
rpmi_molarity(model)
print ('RPMI_molarity', model.slim_optimize())
complete(model)
print('Complete media', model.slim_optimize())

RPMI 1.5748164611774143
MDM 1.6558002724413403
RPMI_molarity 0.28499647807840733
Complete media 2.4998508320953516


In [294]:
#Add exchange reactions for extracellular reactions that are missing, or delete metabolites as appropriate
exchanges=[]
exchanges_mets=[]
extracellular=[]
for metabolite in model.metabolites:
    if metabolite.compartment == 'extracellular':
        extracellular.append(metabolite.id)
    else :
        pass

for reaction in model.reactions:
    if "EX_" in reaction.id:
        exchanges.append(reaction.id)

for x in exchanges:
    for metabolites in model.reactions.get_by_id(x).metabolites:
        exchanges_mets.append(metabolites.id)

In [295]:
for x in extracellular:
    if x in exchanges_mets:
        pass
    else:
        print(x)

ag_e
capsule_NM_e
glcn_e
acald_e


In [298]:
#acald export was previously deleted as it is not detected extracellularly in Gc cultures. removed extracellular acald as a metabolite.  
acald_e = model.metabolites.acald_e
model.remove_metabolites(acald_e)
ACALDtex=model.reactions.ACALDtex
model.remove_reactions(ACALDtex)

#gluconate is not thought to be imported/used as a carbon source by neisseria (https://www.karger.com/Article/FullText/447093)
glcn_e=model.metabolites.glcn_e
model.remove_metabolites(glcn_e)

#delete capsule metabolite, which is not produced by Gc
capsule=model.metabolites.capsule_NM_e
model.remove_metabolites(capsule)

#add exchange reaction for silver
reaction = Reaction('EX_ag_e')
reaction.name = 'EX of Ag_e'
reaction.subsystem = 'Exchange'
reaction.lower_bound = 0
reaction.upper_bound = 0
reaction.annotation = {'sbo': 'SBO:0000627'}
reaction.add_metabolites({model.metabolites.ag_e: -1.0})
reaction.notes['NOTES'] = 'Added exchange reaction that was missing'
model.add_reactions([reaction])

In [299]:
#delete orphan metabolties left over from reconstruction curation process
orphan_mets=[]
for metabolite in model.metabolites:
    if len(metabolite.reactions) == 0:
        orphan_mets.append(metabolite.id)

for mets in orphan_mets:
    m=model.metabolites.get_by_id(mets)
    model.remove_metabolites(m)

In [300]:
pro_L_c

Metabolite identifier,pro_L_c
Name,M_pro_L_c
Memory address,0x01783dbba1f0
Formula,C5H9NO2
Compartment,cytoplasm
In 7 reaction(s),"biomass, PROabcpp, P5CR, PROt2rpp, PROt1pp, PROD2, PROHX"


In [301]:
r=model.reactions.AMPTASEPG
pro__L_c = model.metabolites.pro__L_c

r.add_metabolites({
    pro_L_c: 1.0,
    pro__L_c: -1.0})


r

Reaction identifier,AMPTASEPG
Name,Aminopeptidase (pro-gly)
Memory address,0x01783e3a8ee0
Stoichiometry,h2o_c + progly_c --> gly_c + pro_L_c M_h2o_c + L-Prolinylglycine --> M_gly_c + M_pro_L_c
GPR,NGO1227
Lower bound,0.0
Upper bound,1000.0


In [303]:
model.remove_metabolites(pro__L_c)

In [302]:
#This gene catalyzes a bidirection conversion of alanine into pyruvate. However, although the enzyme is technically birectional,
#in e.coli alanine transaminase expretion is supressed in the presence of alanine and the downstream product leucine. 
#Additionally, Gc is not capable of growing on alanine as a carbon source. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2950514/
#As such, this is likely not a catabolic enzyme. I am limiting it to the synthesis of alanine only. 
model.reactions.ALATA_L.upper_bound=0

In [304]:
len(model.genes)

560

In [305]:
#Subtract Sgene and Orphan gene for naming
model.id ='NGO_558'

In [306]:
model

Name,NGO_558
Memory address,0x01783cb76460
Number of metabolites,1263
Number of reactions,1423
Number of groups,0
Objective expression,1.0*biomass - 1.0*biomass_reverse_01e59
Compartments,"periplasm, cytoplasm, extracellular"


In [307]:
cobra.io.write_sbml_model(model, 'C:/Users/Aimee/Documents/UVA/Metabolic_Modeling/organized/models/annotatedGCmodel.xml')

In [308]:
cobra.io.save_json_model(model, 'C:/Users/Aimee/Documents/UVA/Metabolic_Modeling/organized/models/annotatedGCmodel.json')

In [ ]:
#Send model to modelPolisher

#Cd ModelPolisher
#docker-compose run -v C:\Users\Aimee\models:/models/ polisher java -jar /ModelPolisher-2.1-beta.jar --input=/models/NGO_557.sbml --output=/models/output/NGO_557_polished.sbml --annotate-with-bigg=true --add-adb-annotations=true --output-combine=true

#Extract from Zip file (Users-->Aimee-->models)
#Save model to "cwd+'/Gc_GENRE_2022/Curate_Gc_Model/In_progress_curation/NGO_557_polished.sbml"
